## Instalando bibliotecas

In [1]:
# instalando pandera
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importando bibliotecas

In [2]:
# importando bibliotecas
import pandas as pd
import pandera as pa

In [3]:
pd.set_option('display.max_columns',100)
#pd.set_option('display.max_rows')

## Extraindo a base de dados

In [4]:
# extraindo base de dados
df = pd.read_csv('https://storage.googleapis.com/aulas-bc26-dados-gbs/alesp/despesa_alesp.csv')

In [5]:
# vendo se está tudo certo com a base de dados
df.head(3)

,Ano,Matricula,Mes,Valor,CNPJ,Tipo,Fornecedor
0,2013,300098,1,575.53,5.049700e+11,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO VANUTRE LTDA
1,2013,300098,1,1650.76,4.881450e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,S.D.LUIZARI & CIA. LTDA.
2,2013,300098,1,1799.29,6.586291e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO EXECUTIVO LTDA


In [6]:
df.tail(3)

,Ano,Matricula,Mes,Valor,CNPJ,Tipo,Fornecedor
118414,2022,300655,9,599.29,4.639213e+13,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",SECRETARIA MUNICIPAL DA FAZENDA
118415,2022,300655,9,1836.13,6.106391e+13,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",CONDOMINIO EDIF IBIRAPUERA PLAZA CENTER
118416,2022,300655,9,2835.90,2.491558e+12,O - LOCAÇÃO DE VEÍCULO,UNIDAS VEÍCULOS ESPECIAIS S.A.


## Iniciando pré-análise

In [7]:
# vendo se tipos de dados batem com os tipos das colunas
df.info()
# aqui vemos que podemos mudar tipo de dados do ano, mes, cnpj

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118417 entries, 0 to 118416
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Ano         118417 non-null  int64  
 1   Matricula   118417 non-null  int64  
 2   Mes         118417 non-null  int64  
 3   Valor       118417 non-null  float64
 4   CNPJ        116563 non-null  float64
 5   Tipo        118417 non-null  object 
 6   Fornecedor  118417 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 6.3+ MB


In [8]:
#consulta valores ausentes
df.isna().sum()
#inconsistencias encontradas: CNPJS zerados

Ano              0
Matricula        0
Mes              0
Valor            0
CNPJ          1854
Tipo             0
Fornecedor       0
dtype: int64

In [9]:
# vendo os valores de qual coluna são únicos
print(df.Ano.is_unique) 
print(df.Matricula.is_unique) 
print(df.Mes.is_unique)
print(df.Valor.is_unique)
print(df.CNPJ.is_unique)
print(df.Tipo.is_unique)
print(df.Fornecedor.is_unique)
# não podemos usar nenhuma dessas colunas como rótulo, pois todas tem valores repetidos

False
False
False
False
False
False
False


## Tratamento Inicial de Dados

### Back-up

In [10]:
# backup do dataframe original
dfback = df.copy()

In [11]:
dfback.dtypes

Ano             int64
Matricula       int64
Mes             int64
Valor         float64
CNPJ          float64
Tipo           object
Fornecedor     object
dtype: object

### Renomeando Colunas

In [12]:
# renomeando as colunas 
df.rename(columns={
    'Ano':'ano',
    'Matricula':'matricula',
    'Mes':'mes',
    'Valor':'valor',
    'CNPJ':'cnpj',
    'Tipo':'tipo',
    'Fornecedor':'fornecedor'
    }, inplace=True)

In [13]:
df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj          float64
tipo           object
fornecedor     object
dtype: object

### Procurando Mais Inconsistências

In [14]:
# procurando dados inconsistentes
sorted(pd.unique(df['fornecedor']))
# inconsistencias encontradas: nomes diferentes para um mesmo tipo de fornecedor

['1 OFICIAL DE REGISTRO DE IMOVEIS DA COMARCA DE ARARAQUARA',
 '1 OFICIAL DE REGISTRO DE IMOVEIS DE SAO PAULO',
 '1 TABELIAO DE NOTAS DA CAPITAL',
 '1 TABELIONATO DE NOTAS',
 '1 TABELIONATO DE NOTAS E ANEXOS',
 '1010 ARTESANATO EM PIZZAS LTDA',
 '1088 COND.IBIRAPUERA CENTRAL PARK',
 '11º TABELIÃO DE NOTAS',
 '123 MILHAS',
 '123 VIAGENS E TURISMO',
 '123 VIAGENS E TURISMO LTDA',
 '13 A INFORMATICA E MATERIAL DE ESCRITÓRIO LTDA.',
 '14º TABELIONATO DE NOTAS',
 '1857 COMERCEIO E EXPORTAÇÃO DE CAFÉ LTDA',
 '1857 COMERCIO E EXPORTAÇAO DE CAFE',
 '19 CARTORIO DE REGISTRO CIVIL',
 '1º OFICIAL DE REGISTRO DE IMÓVEIS DE MOGI DAS CRUZES-SP',
 '1º TABELIAO DE NOTAS E DE PROTESTOS - BEL. DENILSON DANIEL',
 '1º TABELIAO DE NOTAS E PROTESTO DE SUZANO',
 '2 AMIGOS COM. DE COMBUST. DE MONTE APRAZIVEL LTDA.',
 '2 AMIGOS COMERCIO DE COMBUSTIVEIS LTDA',
 '2 M MARKETING E PLANEJAMENTO LTDA ME',
 '2 OFICIAL DE REGISTRO DE IMOVEIS, TITULOS E DOCUMENTOS E CIVIL DE PESSOA JURIDICA DE GUARULHOS',
 '2 TABELIAO 

## Validando os Dados

In [15]:
df.dtypes

ano             int64
matricula       int64
mes             int64
valor         float64
cnpj          float64
tipo           object
fornecedor     object
dtype: object

In [23]:
# criação do schema de validação dos dados
schema = pa.DataFrameSchema(
    columns = {
      'ano':pa.Column(pa.Int),
      'matricula':pa.Column(pa.Int),
      'mes':pa.Column(pa.Int),
      'valor':pa.Column(pa.Float),
      'cnpj':pa.Column(pa.Float,nullable=True),
      'tipo':pa.Column(pa.String),
      'fornecedor':pa.Column(pa.String)
    }
)

In [24]:
# validador do DF
schema.validate(df)

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
0,2013,300098,1,575.53,5.049700e+11,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO VANUTRE LTDA
1,2013,300098,1,1650.76,4.881450e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,S.D.LUIZARI & CIA. LTDA.
2,2013,300098,1,1799.29,6.586291e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO EXECUTIVO LTDA
3,2013,300098,1,36.40,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
4,2013,300098,1,3100.00,9.904660e+11,"D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS ...",CROMOGRAF GRAFICA DE PRESIDENTE PRUDENTE LTDA ...
...,...,...,...,...,...,...,...
118412,2022,300655,9,1250.00,1.531481e+13,K - LOCAÇÃO DE BENS MÓVEIS,FYIVAS SOLUÇÕES EM TECNOLOGIA LTDA ME
118413,2022,300655,9,2500.00,6.729494e+10,L - LOCAÇÃO DE BENS IMÓVEIS,MARIA INES SINISGALLI
118414,2022,300655,9,599.29,4.639213e+13,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",SECRETARIA MUNICIPAL DA FAZENDA
118415,2022,300655,9,1836.13,6.106391e+13,"M - MANUTENÇÃO DE BENS MÓVEIS, IMÓVEIS, CONDO...",CONDOMINIO EDIF IBIRAPUERA PLAZA CENTER


In [25]:
df.loc[[3,27,77,111]] 

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
3,2013,300098,1,36.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
27,2013,300098,2,10.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
77,2013,300098,3,29.0,NaN,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",TAXI
111,2013,300098,4,203.5,NaN,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",TAXI


In [19]:
#contagem de valores "válidos"
df.count()

ano           118417
matricula     118417
mes           118417
valor         118417
cnpj          116563
tipo          118417
fornecedor    118417
dtype: int64

## Filtros

In [66]:
# para criar um filtro, começamos procurando uma coluna com menos tipos de dados
# para faciliar a análise
sorted(pd.unique(df['tipo']))

['A - COMBUSTÍVEIS E LUBRIFICANTES',
 'C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE VEÍCULOS ; PEDÁGIOS',
 'D - MATERIAIS E SERVIÇOS GRÁFICOS, DE CÓPIAS  E REPRODUÇÃO DE DOCS',
 'E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS DE CONSUMO',
 'F - SERVIÇOS TÉCNICOS PROFISSIONAIS (CONSULTORIA, PESQUISAS ETC)',
 'G - ASSINATURAS DE PERIÓDICOS, PUBLICAÇÕES, INTERNET E SOFTWARES',
 'H - SERV.UTIL.PÚBLICA (TELEF.MÓVEL/FIXA, ENERGIA, ÁGUA, GÁS ETC)',
 'I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO',
 'J - SERVIÇOS DE COMUNICAÇÃO',
 'K - LOCAÇÃO DE BENS MÓVEIS',
 'L - LOCAÇÃO DE BENS IMÓVEIS',
 'M - MANUTENÇÃO DE BENS MÓVEIS,  IMÓVEIS, CONDOMÍNIOS E OUTROS',
 'N - MORADIA',
 'O - LOCAÇÃO DE VEÍCULO',
 'P - DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR']

### Filtro C

In [67]:
# criando um filtro para tipo C
filtroc = df.tipo == 'C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE VEÍCULOS ; PEDÁGIOS'

In [68]:
# cria um df filtrado com as informações da categoria C
dpedagio = df.loc[filtroc]

In [69]:
dpedagio

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
3,2013,300098,1,36.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
26,2013,300098,2,30.0,6.116149e+13,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,combustivel
27,2013,300098,2,10.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
60,2013,300098,3,374.4,1.845323e+12,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,AUTOESTE VEÍCULOS E PEÇAS LTDA.
61,2013,300098,3,230.0,6.553230e+13,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,J. L. AUTO ELÉTRICA LTDA. - ME
...,...,...,...,...,...,...,...
117309,2021,300650,11,115.2,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117374,2022,300650,1,30.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117433,2022,300650,3,53.0,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117590,2022,300650,8,398.9,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio


In [70]:
# cria um filtro para pegar partes de string que podem estar nos dados relacionados a pedagio
filtroped = dpedagio.fornecedor.str.contains('PED|PEG|SEM')

In [71]:
# cria um novo df com base no df filtrado, passando mais um filtro de parametro
dfpedagio = dpedagio.loc[filtroped]

In [72]:
dfpedagio

,ano,matricula,mes,valor,cnpj,tipo,fornecedor


In [73]:
# encontra dos nomes de fornecedor que querem dizer a mesma coisa
sorted(pd.unique(dfpedagio['fornecedor']))

[]

In [74]:
# compatibilizando dados de fornecedor para todos ficarem iguais
df.replace(['CUPONS DE PEDÁGIO',
            'PEDAGIO',
            'PEDAGIO DE ELISIARIO',
            'PEDÁGIO',
            'PEDÁGIOS',
            'PEGAGIO',
            'PRAÇA DE PEDÁGIO DE GUARUJÁ',
            'SEM PARAR',
            'SEM PARAR - CENTRO DE GESTÃO DE MEIOS DE PGTO. LTDA.',
            'SEM PARAR CENTRO DE GESTÃ&#131;O DE MEIOS DE PGTO. LTDA',
            'SEM PARAR INSTITUIÇÃO DE PAGAMENTO LTDA',
            'SEM PARAR INSTITUIÇÃO DE PAGAMENTO LTDA.'], 
            'pedagio', inplace=True)

In [75]:
ftpedagio = df.fornecedor == 'pedagio'
dtfpedagio = df.loc[ftpedagio]
dtfpedagio

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
3,2013,300098,1,36.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
27,2013,300098,2,10.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
97,2013,300098,4,38.6,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
128,2013,300098,5,5.2,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
161,2013,300098,6,13.7,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
...,...,...,...,...,...,...,...
117270,2021,300650,10,75.6,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117309,2021,300650,11,115.2,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117374,2022,300650,1,30.4,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio
117433,2022,300650,3,53.0,NaN,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,pedagio


### Filtro I

In [76]:
# criando um filtro para o tipo I
filtroi = df.tipo == 'I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCOMOÇÃO'

In [77]:
# criando um df filtrado com informações apenas do tipo I
dhospedagem = df.loc[filtroi]

In [78]:
dhospedagem

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
14,2013,300098,1,3824.11,6.164253e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",GOL LINHAS AREAS INTELIGENTES
15,2013,300098,1,193.93,6.476508e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHOPERIA E LANCHONETE H2 LTDA - EPP
42,2013,300098,2,18.00,2.530487e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",INSTITUTO MENSAGEIROS
43,2013,300098,2,2377.43,6.164253e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",GOL LINHAS AREAS INTELIGENTES
44,2013,300098,2,139.48,6.476508e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHOPERIA E LANCHONETE H2 LTDA - EPP
...,...,...,...,...,...,...,...
118405,2022,300655,9,73.50,7.313854e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",RESTAURANTE PAPAGAIUS LTDA
118406,2022,300655,9,190.00,1.132508e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PER TE AMORE PIZZAS LTDA
118407,2022,300655,9,16.60,1.700949e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",GRAO DO IPIRANGA PAES E DOCES LTDA
118408,2022,300655,9,819.00,2.919185e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PERSONALITE ADMINISTRADORA DE HOTEIS E CONDOMI...


In [79]:
# criando um filtro para procurar dados da coluna fornecedor que tenham HOT no nome
filtrohot = dhospedagem.fornecedor.str.contains('HOT')

In [80]:
# criando um df com base no df filtrado pro tipo I 
# e passando de parametro o filtro para procurar strings que contenham HOT
dfhospedagem = dhospedagem.loc[filtrohot]

In [81]:
dfhospedagem

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
1909,2022,300098,2,196.00,1.061991e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CASTOR ADMINISTRACAO DE HOTELARIA LTDA
1910,2022,300098,2,225.63,3.471411e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",HD PLAZA HOTEL LTDA EPP
1911,2022,300098,2,248.17,9.632945e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",JR PARK HOTEL LTDA.
1934,2022,300098,3,315.00,3.471411e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",HD PLAZA HOTEL LTDA EPP
1936,2022,300098,3,468.00,9.632945e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",JR PARK HOTEL LTDA.
...,...,...,...,...,...,...,...
118336,2022,300655,7,1080.00,5.211653e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",JR MARILIA HOTEIS EIRELI-EPP
118337,2022,300655,7,1638.00,2.919185e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",PERSONALITE ADMINISTRADORA DE HOTEIS E CONDOMI...
118338,2022,300655,7,1212.00,3.430618e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",SMART ADMINISTRADORA HOTELEIRA LTDA
118369,2022,300655,8,1085.76,5.949818e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",LE PREVILEGE ADMINISTRADORA DE HOTEIS E CONDOM...


In [82]:
# encontra dos nomes de fornecedor que querem dizer a mesma coisa
sorted(pd.unique(dfhospedagem['fornecedor']))

['2.0 HOTEIS BAURU',
 '2.0 HOTEIS BAURU LTDA',
 '2.0 HOTEIS BIRIGUI LTDA',
 '2.0 HOTEIS FRANCA LTDA',
 '2.0 HOTEIS RIBEIRAO PRETO LTDA.',
 '2.0 HOTEIS RIO PRETO LTDA',
 '2.O HOTÉIS PIRACICABA LTDA',
 '994502-2.0HOTEIS ARAÇATUBA LTDA',
 'A FAZENDA HOTEIS E CONVENÇOES LTDA',
 'AB2 SERVICOS DE HOTELARIA EIRELI',
 'ABACO HOTELARIA E SERVICOS S.A.',
 'ABACO HOTELARIA E SERVIÇO S.A',
 'ABACO HOTELARIA E SERVIÇOS S.A',
 'ABAPORU HOTEIS LTDA',
 'ABAPORU HOTEIS LTDA - ME',
 'ABAPORU HOTEIS LTDA ME',
 'AGUA BRANCA PARK HOTEL EIRELI',
 'AGUERA HOTEL LTDA',
 'AGUILLAR & SOUZA HOTEL LTDA - ME',
 'ALFONSO HOTEL LTDA ME',
 'ALFONSOS HOTEL LTDA ME',
 'ALJA HOTELARIA & SERVIÇOS LTDA',
 'ALVES HOTEL LTDA - EPP',
 'ALVES HOTEL LTDA ME EPP',
 'ALVORADA HOTEL DE NOVO HORIZONTE LTDA ME',
 'AMANDALINE HOTEL LTDA',
 'ANGELO PALACE HOTEL LTDA',
 'ANNA HOTEL LTDA',
 'ANTONIO CESAR LOPES HOTEL EIRELI',
 'ANTONIOLLI HOTEIS E TURISMO LTDA',
 'APART HOTEL OLAVO BILAC LTDA - ME',
 'APART HOTEL OLAVO BILAC LTDA ME',


In [83]:
# compatibilizando dados de fornecedor
df.replace(['2.0 HOTEIS BAURU',
            '2.0 HOTEIS BAURU LTDA',
            '2.0 HOTEIS BIRIGUI LTDA',
            '2.0 HOTEIS FRANCA LTDA',
            '2.0 HOTEIS RIBEIRAO PRETO LTDA.',
            '2.0 HOTEIS RIO PRETO LTDA',
            '2.O HOTÉIS PIRACICABA LTDA',
            '994502-2.0HOTEIS ARAÇATUBA LTDA',
            'A FAZENDA HOTEIS E CONVENÇOES LTDA',
            'AB2 SERVICOS DE HOTELARIA EIRELI',
            'ABACO HOTELARIA E SERVICOS S.A.',
            'ABACO HOTELARIA E SERVIÇO S.A',
            'ABACO HOTELARIA E SERVIÇOS S.A',
            'ABAPORU HOTEIS LTDA',
            'ABAPORU HOTEIS LTDA - ME',
            'ABAPORU HOTEIS LTDA ME',
            'AGUA BRANCA PARK HOTEL EIRELI',
            'AGUERA HOTEL LTDA',
            'AGUILLAR & SOUZA HOTEL LTDA - ME',
            'ALFONSO HOTEL LTDA ME',
            'ALFONSOS HOTEL LTDA ME',
            'ALJA HOTELARIA & SERVIÇOS LTDA',
            'ALVES HOTEL LTDA - EPP',
            'ALVES HOTEL LTDA ME EPP',
            'ALVORADA HOTEL DE NOVO HORIZONTE LTDA ME',
            'AMANDALINE HOTEL LTDA',
            'ANGELO PALACE HOTEL LTDA',
            'ANNA HOTEL LTDA',
            'ANTONIO CESAR LOPES HOTEL EIRELI',
            'ANTONIOLLI HOTEIS E TURISMO LTDA',
            'APART HOTEL OLAVO BILAC LTDA - ME',
            'APART HOTEL OLAVO BILAC LTDA ME',
            'APIACAS HOTEL LTDA',
            'APRT HOTEL OLAVO BILAC LTDA  ME',
            'APS HOTEL LTDA  ME',
            'AR & M PALACE HOTEL LTDA',
            'ARCA HOTEL LTDA ME',
            'ARCO EXPRESS HOTEL LTDA',
            'ARCO HOTEIS ECONOMICOS LTDA.',
            'ARCO HOTEIS INTELIGENTES LTDA',
            'ARUA HOTEL LTDA',
            'ARUA HOTEL LTDA EPP',
            'ARUÁ - HOTEL LTDA - EPP',
            'ASAKUSA HOTEIS LTDA',
            'ASSET ATIVIDADES HOTELEIRAS LTDA',
            'ASSI PALACE HOTEL LTDA',
            'ASTORGA HOTEL LTDA',
            'ASTRO HOTEL LTDA',
            'ATA HOTEIS E TURISMO LTDA-EPP',
            'ATHANASE SARANTOPOULOS HOTEIS E TURISMO',
            'ATHANASE SARANTOPOULOS HOTEIS E TURISMO LTDA',
            'ATHANASE SARANTOPOULOS HOTÉIS E TURISMO LTDA',
            'ATIQUE HOTEIS LTDA - EPP',
            'ATIQUE HOTEIS LTDA EPP',
            'ATLANTIA HOTELS INTERNATIONAL (BRASIL) LTDA',
            'ATLANTICA HOTELS INTERNACIONAL BRASIL LTDA',
            'ATLANTICA HOTELS INTERNACTIONAL BRASIL LTDA',
            'ATLANTICA HOTELS INTERNATIONAL BRASIL',
            'ATLANTICA HOTELS INTERNATIONAL BRASIL LTDA',
            'ATLANTICA HOTELS INTERNATIONAL BRASIL LTDA.',
            'ATLANTICA HOTRELS INTERNATIONAL BRASIL LTDA',
            'ATLANTICO GOLDEN APART HOTEL LTDA',
            'ATRIO HOTEIS',
            'ATRIO HOTEIS S A',
            'ATRIO HOTEIS S.A',
            'ATRIO HOTEIS S.A.',
            'ATRIO HOTEIS S.A. IBIS HOTELS',
            'ATRIO HOTEIS S/A',
            'ATRIO HOTEIS SA',
            'ATRIO HOTEIS SA.',
            'ATRIO HOTEL S/A',
            'ATRIOS HOTEL S A',
            'ATTILIOS APRT HOTEL LTDA',
            'AVARE PLAZA HOTEL PLUS LTDA',
            'AVARE PLAZA HOTEL SLIM LTDA',
            'AVENIDA HOTEL SS LTDA ME',
            'AVENIDA HOTELS S LTDA ME',
            'B.A. RAYMUNDO HOTEL ME',
            'BARCELONA HOTEL SC LTDA',
            'BAURU OFFICE HOTEL LTDA ME',
            'BC DA ROCHA HOTEL EIRELLI ME',
            'BCA TURISMO E HOTELARIA LTDA ME',
            'BCA TURISMO E HOTELARIA LTDA.- ME',
            'BEIRA RIO HOTELARIA DE TURISMO LTDA',
            'BEIRA RIO HOTELARIA DE TURISMO LTDA - ME',
            'BEIRA RIO PALACE HOTEL LTDA',
            'BEKASSIN BOTUCATU HOTEIS LTDA',
            'BIAZI PLAZA HOTEL LTDA',
            'BIAZI PLAZA HOTEL LTDA.',
            'BL HOTEL DO LADO LTDA ME',
            'BLACK STREAM HOTEL LTDA - EPP',
            'BLACK STREAM HOTEL LTDA- EPP',
            'BOA TURISMO E HOTELARIA LTDA ME',
            'BONAPARTE HOTEL RESIDENCE - HOTEIS',
            'BRASIL 21 EVENTOS E HOTELARIA LTDA',
            'BRISAMAR PRAIA HOTEL LTDA EPP',
            'BRISTOL HOTEL LTDA',
            'BT BAURU HOTEIS',
            'BT BAURU HOTEIS LTDA',
            'BT FARIA LIMA HOTEIS LTDA.',
            'BT VALINHOS HOTEIS LTDA',
            'BT VALINHOS HOTÉIS LTDA',
            'C A HOTELARIA LTDA',
            'C D A DA SILVA HOTEL ME',
            'C.C. DE MENDONÇA HOTEL - EPP',
            'C.C.DEMENDONÇA HOTEL EPP',
            'CALANDRE HOTEL LTDA ME',
            'CALI HOTEL CASTILHO LTDA',
            'CALIGIO HOTEL EIRELI - ME',
            'CAMPESATO E RODRIGUES HOTEL LTDA ME',
            'CAMPINAS AEROPORTO HOTEL LTDA',
            'CAMPUS HOTEL ELOAH LTDA - ME',
            'CANHEO & PEREIRA SERV. DE HOTELARIA LTDA - EPP',
            'CANHEO & PEREIRA SERVIÇOS DE HOTELARIA LTDA -EPP',
            'CANHOTO & CANHADO RESTAURANTES LTDA - ME',
            'CARDUM PALACE HOTEL LTDA EPP',
            'CARNEIRO DE MELO HOTEIS EIRELI EPP',
            'CARVALHO E RODRIGUES HOTEL LTDA',
            'CASA GRANDE HOTEL S/A',
            'CASA GRANDE HOTEL SA',
            'CASA GRANDE OLIMPIA HOTEL LTDA',
            'CASA GRANDE PLIMPIA HOTEL LTDA',
            'CASAS NOVAS HOTEL EIRELI ME',
            'CASTOR - ADMINISTRACAO DE HOTELARIA LTDA',
            'CASTOR ADMINISTRACAO DE HOTELARIA LTDA',
            'CASTOR ADMINISTRAÇAO DE HOTELARIA LTDA',
            'CASTOR ADMINISTRAÇÃO DE HOTELARIA LTDA',
            'CAZUZA PALACE HOTEL DE TUPA LTDA',
            'CAZUZA PALACE HOTEL DE TUPA LTDA ME',
            'CENTRAL PLAZA DE TUPA HOTEL LTDA ME',
            'CENTRAL PLAZA DE TUPÃ HOTEL LTDA ME',
            'CENTRO COMERCIAL ANTICO PLAZA HOTEL LTDA EPP',
            'CHALÉ BRASIL HOTEL LTDA',
            'CHIQUINHO SERVIÇOS DE HOTELARIA LTDA - ME',
            'CHIQUINHO SERVS DE HOTELARIA',
            'CIVITAS COMPLEXO SUL HOTEIS  E CONDOMINIOS LTDA.',
            'CIVITAS COMPLEXO SUL HOTEIS E CONDOMINIOS LDTA',
            'CIVITAS COMPLEXO SUL HOTEIS E CONDOMINIOS LTDA',
            'CLAMAIS HOTELARIA LTDA ME',
            'CLASS HOTEL',
            'CLAUDIO BISPO HOTEIS EIRELI',
            'CLEZIO DE PAULA GALDIANO-ME - HOTEL MARAJOARA',
            'COLONIAL HOTEL CATANDUVA LTDA - ME',
            'COMFORT HOTEL SERTÃOZINHO',
            'CONFIANCE HOTEL & LAZER LTDA',
            'CONFIANCE HOTEL E LAZER LTDA',
            'CONSTRUSOLA CONSTRUÇAO E HOTEL LTDA',
            'CRETA EXECUTIVE HOTEL LTDA',
            'CRRINI HOTELARIA LTDA',
            'D MARCO HOTEL LTDA ME',
            'D.M. SISMEIRO HOTEL ME',
            'DAN INN HOTEL ARARAQUARA',
            'DAN INN HOTEL ARARAQUARA LTDA',
            'DAN INN HOTEL FRANCA LTDA',
            'DAN INN HOTEL FRANCA LTDA ME',
            'DAN INN HOTEL FRANCA LTDA.',
            'DAN INN HOTEL LTDA',
            'DAN INN HOTEL RIBEIRAO PRETO LTDA',
            'DAN INN HOTEL RIBEIRÃO PRETO LTDA',
            'DAV INN HOTEL FRANCA LTDA',
            'DOM APART HOTEL LORENA LTDA ME',
            'DUMONT PARK HOTEL LTDA ME',
            'E.S. GINIS - OLIMPIA HOTEL',
            'EBH HOTEL LTDA',
            'ECONOMIC INN HOTEL LDA EPP',
            'ECONOMIC INN HOTEL LTDA.',
            'EMPREENDI. HOTEL E TURIST BANDEIRAS LTDA - ANGATU HOTEL',
            'EMPREENDIMENTO HOTELEIRO RIO PRETO LTDA',
            'EMPREENDIMENTOS HOTELEIROS E TUR BANDEIRAS LTDA',
            'EMPREENDIMENTOS HOTELEIROS E TURISTICOS BANDEIRAS LTDA',
            'EMPRESA RIOPRETENSE DE HOTEIS E TURISMO LTDA',
            'EMPRESA RIOPRETENSE DE HOTÉIS E TURISMO LTDA',
            'ERDAL HOTELARIA LTDA',
            'ESTANCIA AVARE HOTEL LTDA',
            'ESTANCIA AVARE HOTEL LTDA ME',
            'ESTORIAL APART HOTEL LTDA ME',
            'ESTORIL APART HOTEL',
            'ESTORIL APART HOTEL LTDA - ME',
            'ESTORIL APART HOTEL LTDA ME',
            'ESTORILAPART HOTEL LTDA ME',
            'EURO SUIT HOTEL CAMPINAS LTDA',
            'EUROPA HOTEL',
            'EXCELLENCE PLAZA HOTEL LTDA ME',
            'EXCLUSIVA PROPRIETARIA DE HOTEL LTDA',
            'EZ HOTEIS LTDA',
            'FARO SAO TOME HOTE LTDA',
            'FARO SAO TOME HOTEL LTDA',
            'FARO SAO TOME HOTEL LTDA.',
            'FARO SÃO TOME HOTEL LTDA',
            'FENIX HOTEL CAMPINAS EIRELI - EPP',
            'FENIX HOTEL CAMPINAS EIRELI EPP',
            'FENIX HOTEL CAMPINAS EIRELI-EPP',
            'FERIAN PLAZA HOTEL LTDA. EPP',
            'FILDI HOTEL LTDA',
            'FIRENZE HOTEL EIRELI',
            'FIRENZE HOTEL EIRELI ME',
            'FLORENCA HOTEL LTDA',
            'FLORIDA HOTEL PERUÍBE NELSI CARLOT ME',
            'FRAGATA APART HOTEL LTDA EPP',
            'FREITAS & DONADI PARK HOTEL LTDA ME',
            'GABHEN FERNANDEZ HOTEL LTDA',
            'GABIRA HOTEL EIRELE ME',
            'GARDEN HOTELS EMPREENDIMENTOS LTDA',
            'GARPELLI & BATTAGLINI LTDA - VALE VERDE PALACE HOTEL',
            'GATTIS HOTEL LTDA',
            'GEPE EMPREENDIMENTO DE HOTELARIA LTDA',
            'GEPE EMPREENDIMENTOS DE HOTELARIA LTDA',
            'GEPE EMPREENDIMENTOS DE HOTELARIA LTDA.',
            'GERALDELLI CENTER HOTEL LTDA',
            'GERALDELLI PARK HOTEL LTDA-ME',
            'GMENDES HOTEIS E PARTICIPAÇÕES LTDA',
            'GMENDES HOTEIS E PARTICIPÇOES LTDA',
            'GONÇALVES & SISMEIRO HOTEL LTDA',
            'GONÇALVES E SISMEIRO HOTEL LTDA',
            'GRALHA AZUL HOTEL LTDA ME',
            'GRAN CONTINENTAL HOTEL LTDA',
            'GRAN HOTEL MINA LTDA',
            'GRAN HOTEL MORADA DO SOL',
            'GRAN HOTEL NINA LTDA',
            'GRAN RIO HOTEL DAN INN LTDA',
            'GRAN RIO PARK HOTEL LTDA',
            'GRAN TOUR HOTEL DE CAPIVARI LTDA',
            'GRAND BITTAR - LB1 HOTELARIA EIRELI',
            'GRAND HOTEL ROYAL SOROCABA LTDA',
            'GRANDE HOTEL CUITELO LTDA.',
            'GRANDE HOTEL TONON LTDA',
            'GRANDES LAGOS PARK HOTEL',
            'GRANDES LAGOS PARK HOTEL LTDA - ME',
            'GRINBERG EMPREENDIMENTOS HOTELEIROS LTDA',
            "GRINGO'S HOTEL E RESTAURANTE LTDA",
            "GRINGO'S HOTEL E RESTAURANTE LTDA.",
            'GRINGOS HOTEL E RESTAURANTE LTDA ME',
            'GRINGOS S HOTEL E RESTAURANTE LTDA ME',
            'GRINGO´S HOTEL E RESTAURANTE LTDA - ME',
            'GRINGO´S HOTEL E RESTAURANTE LTDA-ME',
            'H DE M JIMENEZ HOTELARIA - EPP',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA - SCP ATHOS BULCÃO',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA - SCP ATHOS BULCÃO - HPLUS ATHOS BULCAO',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA - SCP CULLINAN',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA - SCP FUSION',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA - SCP VISION',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA SCP CULLINAN',
            'H PLUS ADMINISTRAÇÃO E HOTELARIA LTDA SCP FUSION',
            'H PLUS ADMINSTRAÇÃO E HOTELARIA LTDA',
            'H.T.S. EMPREENDIMENTOS HOTELEIROS LTDA- EPP',
            'HABITAT HOTEL DE LEME LTDA',
            'HABITAT HOTEL DE PIRASSUNUNGA LTDA EPP',
            'HABITAT PRIME HOTEL EPP',
            'HC HOTEL LTDA',
            'HD PLAZA HOTEL EPP',
            'HD PLAZA HOTEL LTDA - EPP',
            'HD PLAZA HOTEL LTDA APP',
            'HD PLAZA HOTEL LTDA EPP',
            'HD PLAZA HOTEL LTDA-EPP',
            'HELOU HOTELARIA EIRELI ME',
            'HERCULANO GOMES RIBEIRO FILHO (TABOCAS HOTEL)',
            'HI ARARAQUARA HOTEIS E CONDOMINIOS LTDA',
            'HI ARARAQUARA HOTÉIS E CONDOMÍNIOS LTDA',
            'HI CATANDUVA HOTEIS E COND. LTDA',
            'HI CATANDUVA HOTEIS E CONDOMINIOS LTDA',
            'HI MARILIA HOTEIS E CONDOMINIOS LTDA',
            'HI MARILIA HOTEIS E CONDOMÍNIO LTDA',
            'HI RIBEIRAO PRETO HOTEIS E CONDOMINIOS LTDA',
            'HI RIBEIRÃO PRETO HOTEIS E CONDOMINIOS LTDA',
            'HI SAO JOSE DO RIO PRETO HOTEIS E CONDOMINIOS LTDA',
            'HI SÃO JOSÉ DO RIO PRETO HOTEIS E CONDOMINIOS',
            'HI SÃO JOSÉ DO RIO PRETO HOTEIS E CONDOMINIOS LTDA',
            'HIB ADMINISTRATORA DE HOTEIS E CONDOMINIOS LTDA',
            'HIB HOTEIS E CONDOMINIOS LTDA',
            'HIJ HOTEIS E CONDOMINIOS LTDA',
            'HIJ HOTÉIS E CONDOMÍNIOS LTDA',
            'HIS HOTEIS E CONDOMINIOS LTDA',
            'HIS HOTEL E CONDOMINIOS LTDA',
            'HITS PANTANAL HOTEL',
            'HN HOTEIS E TURISMO LTDA',
            'HOTEIS DE TURISMO SA HOTEISTUR',
            'HOTEIS DE TURISMO SA- HOTEISTUR',
            'HOTEIS EVEREST S.A.',
            'HOTEIS FRANCA LTDA',
            'HOTEIS RIBEIRAO PRETO LTDA',
            'HOTEIS ROYAL PALM PLAZA LTDA',
            'HOTEIS SAO CARLOS LTDA',
            'HOTEL ACCOR BRASIL S/A',
            'HOTEL ACCOR BRASIL SA',
            'HOTEL AGAPITO EIRELI',
            'HOTEL ALJI SALTO LTDA.',
            'HOTEL AQUARIUS DE MARILIA LTDA',
            'HOTEL ARARAQUARA SUITES LTDA',
            'HOTEL ARAÇARI EIRELI - EPP',
            'HOTEL ARAÇATUBA PLAZA EIRELI - EPI',
            'HOTEL ARAÇATUBA PLAZA LTDA',
            'HOTEL AREIA BRANCA LTDA',
            'HOTEL AREIA DOURADA LTDA',
            'HOTEL ARRASTAO LTDA',
            'HOTEL ARUJÁ LTDA',
            'HOTEL ASSAY LTDA ME',
            'HOTEL BARRETOS EIRELI',
            'HOTEL BEIRA RIO S/C LTDA',
            'HOTEL BELA VISTA VOTORANTIM LTDA-ME',
            'HOTEL BELLAVISTA LTDA.',
            'HOTEL BIMD EIRELI - ME',
            'HOTEL BIRIGUI PALACE LTDA - EPP',
            'HOTEL BIRIGUI PALACE LTDA EPP',
            'HOTEL BOCAIUVA LTDA',
            'HOTEL BRASIL',
            'HOTEL BRISA RIO DE JACAREÍ LTDA',
            'HOTEL C.A. DE MARILIA EIRELI',
            'HOTEL CAMARGO LTDA ME',
            'HOTEL CAMARGO LTDA.-ME',
            'HOTEL CAPRICCIO II LTDA',
            'HOTEL CASTRO MENDES LTDA',
            'HOTEL CAZOTTI & VASCONCELLOS LT EPP',
            'HOTEL CENTRAL LINS',
            'HOTEL CHAILLOT EIRELI EPP',
            'HOTEL CHALES GRAMADO',
            'HOTEL CHAMONIX ARAÇATUBA LTDA',
            'HOTEL CHAMONIX RECREIO LTDA EPP',
            'HOTEL CIDADE DE MOGI MIRIM LTDA -ME',
            'HOTEL COLINAS LTDA',
            'HOTEL COLONIAL PORTO FERREIRA LTDA - ME',
            'HOTEL COLUMBIA BTU LTDA',
            'HOTEL COLUMBIA PIRASSUNUNGA LTDA',
            'HOTEL COLUMBIA PIRASSUNUNGA LTDA - EPP',
            'HOTEL COQUILLE LTDA ME',
            'HOTEL CORDILHEIRA LTDA',
            'HOTEL COSTA NORTE MASSAGUAÇU LDTA - ME',
            'HOTEL CRISTAL PALACE DE LINS LTDA',
            'HOTEL DAN INN BARRETOS LTDA',
            'HOTEL DAN INN PREMIUM LTDA',
            'HOTEL DAN INN SAO CARLOS LTDA',
            'HOTEL DAN INN SAO JOSE LTDA - EPP',
            'HOTEL DAN INN SÃO CARLOS LTDA',
            'HOTEL DAN NINN SAO JOSE LTDA EPP',
            'HOTEL DANI LTDA ME',
            'HOTEL DANN INN BARRETOS LTDA',
            'HOTEL DAS DRACENAS LTDA',
            'HOTEL DECK RIO LTDA',
            'HOTEL DEL FIOL LTDA',
            'HOTEL DEL FIOL LTDA.',
            'HOTEL DI GIULIO',
            'HOTEL DRACENAS LTDA.',
            'HOTEL E CHURRASCARIA 41 LTDA - EPP',
            'HOTEL E POUSADA BELA MANHÃ',
            'HOTEL E RESTAURANTE ADOLFO IRMAOS RUSSA LTDA ME',
            'HOTEL E RESTAURANTE CORONA ME',
            'HOTEL E RESTAURANTE PILAO',
            'HOTEL E RESTAURANTE PILÃO',
            'HOTEL E RESTAURANTE PILÃO LTDA.ME',
            'HOTEL E RESTAURANTE RECANTO ELDORADO',
            'HOTEL E RESTAURANTE RIOPEDRENSE LTDA',
            'HOTEL E RESTAURANTE UNIAO LTDA - ME',
            'HOTEL E RESTAURANTE UNIÃO LTDA - ME',
            'HOTEL E RESTAURANTE UNIÃO LTDA ME',
            'HOTEL E RESTAURANTE WAIKIKI LTDA',
            'HOTEL E. F. FIUSA LTDA - ME',
            'HOTEL E.F. FIUSA LTDA',
            'HOTEL E.F.FIUSA LTDA.-ME',
            'HOTEL ELIGI LTDA ME',
            'HOTEL ELITHI S/S LTDA-ME',
            'HOTEL ESPLANADA DE PROMISSAO LTDA',
            'HOTEL F S GIORDANO LTDA EPP',
            'HOTEL FAROL LTDA ME',
            'HOTEL FARTURA PLAZA LTDA - ME',
            'HOTEL FERNANDOPOLIS S.A',
            'HOTEL FERNANDOPOLIS SA',
            'HOTEL FLORENCA LTDA',
            'HOTEL FORMULA 1 EIRELI ME',
            'HOTEL FRAQUITO LTDA',
            'HOTEL FREDY LTDA - EPP',
            'HOTEL GALDIANO NOGUEIRA LTDA - ME',
            'HOTEL GALDIANO NOGUEIRA LTDA ME',
            'HOTEL GALDINO NOGUEIRA LTDA - ME',
            'HOTEL GLOBO RIO LTDA',
            'HOTEL GRAN VALLE LTDA ME',
            'HOTEL GRAUNAS LTDA',
            'HOTEL GUARDA MOR LTDA',
            'HOTEL IBIS BUDGET',
            'HOTEL IBIZA DE ITAPEVA LTDA',
            'HOTEL IBIZA DE ITAPEVA LTDA ME',
            'HOTEL IBIZA DE ITAPEVA LTDA-ME',
            'HOTEL ILHAS DA GRECIA LTDA',
            'HOTEL IPORA ITAPETININGA LTDA EPP',
            'HOTEL IRMAOS RAHAL LTDA ME',
            'HOTEL JAU LTDA',
            'HOTEL JAU LTDA - EPP',
            'HOTEL JWF LIMEIRA LTDA',
            'HOTEL LAREIRA LTDA - ME',
            'HOTEL LETOH MOCOCA LTDA EPP',
            'HOTEL LETOH MOCOCA LTDA-EPP',
            'HOTEL LICATELLI LTDA ME',
            'HOTEL LINDA POUSADA DE SALTO LTDA - ME',
            "HOTEL LIRIO D'AGUA LTDA - EPP",
            'HOTEL LITORAL NORTE EIRELI ME',
            'HOTEL LUCATELLI LTDA ME',
            'HOTEL MARBOR',
            'HOTEL MARCO INTERNACIONAL S/A',
            'HOTEL MARINHO LTDA',
            'HOTEL MARQUES LTDA',
            'HOTEL MATAO SUITES LTDA',
            'HOTEL MAX PLAZA DE MARILIA LTDA',
            'HOTEL MAX PLAZA DE MARÍLIA LTDA',
            'HOTEL MELIA BRASIL XXI',
            'HOTEL MONREALE CLASSIC CAMPINAS LTDA',
            'HOTEL MONREALE RIBEIRAO PRETO LTDA',
            'HOTEL MONREALE RIBEIRÃO PRETO LTDA',
            'HOTEL MORADA DO SOL LTDA - ME',
            'HOTEL NACIIONAL DE RIO PRETO LTDA',
            'HOTEL NACIONAL DE RIO PRETO LTDA',
            'HOTEL NACIONAL DO BRASIL LTDA',
            'HOTEL NACIONAL DUTRA',
            'HOTEL NACIONAL DUTRA LTDA',
            'HOTEL NACIONAL INN ARACATUBA LTDA',
            'HOTEL NACIONAL INN ARAÇATUBA LTDA',
            'HOTEL NACIONAL INN C J LTDA',
            'HOTEL NACIONAL INN CAMPINAS LTDA',
            'HOTEL NACIONAL INN LTDA EPP',
            'HOTEL NACIONAL INN RIBEIRAO PRETO LTDA',
            'HOTEL NACIONAL INN RIBEIRÃO PRETO LTDA',
            'HOTEL NACIONAL INN SAO CARLOS LTDA',
            'HOTEL NACIONAL INN SOROCABA LTDA',
            'HOTEL NACIONAL INN SÃO CARLOS LTDA',
            'HOTEL NACIONAL INN SÃO CARLOS LTDA.',
            'HOTEL NACIONAL RIO PRETO LTDA',
            'HOTEL NACIONAL RIO PRETO LTDA.',
            'HOTEL NACIONAL S.A',
            'HOTEL NACIONAL S/A',
            'HOTEL NETWORK MARKETING SALES EIRELI',
            'HOTEL NINA  LTDA',
            'HOTEL NINA LTDA',
            'HOTEL NINÁ LTDA',
            'HOTEL NOVO HORIZONTE LTDA',
            'HOTEL NOVO HORIZONTE LTDA ME',
            'HOTEL NOVO LIMEIRA',
            'HOTEL NOVO LIMEIRA EIRELI EPP',
            'HOTEL O ESTRADEIRO PRESIDENTE VENCESLAU LTDA EPP',
            'HOTEL ORION JWF ITATIBA LTDA ME',
            'HOTEL PALACE CRISTAL',
            'HOTEL PASSARIM LTDA',
            'HOTEL PHENICIA',
            'HOTEL PHENICIA LTDA',
            'HOTEL PHENICIA LTDA GRAND BITTAR',
            'HOTEL PLAZA LTDA EPP',
            'HOTEL POLO CUESTA LTDA EPP',
            'HOTEL PORTAL D OESTE LTDA',
            'HOTEL PORTAL D OSTE LTDA',
            "HOTEL PORTAL D'OESTE LTDA",
            "HOTEL PORTAL D'OESTE LTDA.",
            'HOTEL PORTAL DÓESTE LTDA',
            'HOTEL PORTO DO RIBEIRÃO',
            'HOTEL POUSADA BARRAVENTO LTDA ME',
            'HOTEL POUSADA CAMPOS VERDEI LTDA ME',
            'HOTEL POUSADA E RESTAURANTE JRC LTDA ME',
            'HOTEL POUSADA GAROUPAS',
            'HOTEL POUSADA MARAZUL',
            'HOTEL POUSADA OURINHOS',
            'HOTEL POUSADA TAQUARIVAÍ LTDA.- EPP',
            'HOTEL PREMIUM FLAT OURINHOS EIRELI',
            'HOTEL PREMIUM NORTE LTDA',
            'HOTEL PRIMAVERA DE ARAMINA LTDA',
            'HOTEL PRIMAVERA IRMÃOS SILVEIRA LTDA.',
            'HOTEL PRUDENTAO LTDA',
            'HOTEL PRUDENTAO LTDA ME',
            'HOTEL PRUDENTÃO LTDA',
            'HOTEL PRUDENTÃO LTDA ME',
            'HOTEL PRUDENTÃO LTDA-ME',
            'HOTEL RAFAELI EIRELI',
            'HOTEL RAFELI EIRELI',
            'HOTEL REAL DE MARACAI LTDA',
            'HOTEL REAL MARACAI LTDA',
            'HOTEL RECANTO DOS PASSAROS DE SÃO SEBASTIÃO LTDA - EPP',
            'HOTEL REDENTODA PLAZA LTDA',
            'HOTEL REGINA C BONITO LTDA / ME',
            'HOTEL REGINA C BONITO LTDA/ME',
            'HOTEL RESORT AGUA DAS ARARAS LTDA',
            'HOTEL ROMANINI & PEREIRA LTDA',
            'HOTEL ROMANINI E PEREIRA LTDA',
            'HOTEL ROTA DO PANTANAL',
            'HOTEL ROTA DO SOL LTDA',
            'HOTEL SABIONE LTDA',
            'HOTEL SAINT MARTIN LIMITADA EPP',
            'HOTEL SANTO SONO LTDA - ME',
            'HOTEL SANTOS',
            'HOTEL SAO MARCOS',
            'HOTEL SAO MARCOS ORLANDIA',
            'HOTEL SERRA DA ESTRELA LTDA-ME',
            'HOTEL SOLARIS LTDA ME',
            'HOTEL SONHO MEU LTDA ME',
            'HOTEL STELATI LTDA.',
            'HOTEL SÃO CHARBEL',
            'HOTEL SÃO CHARBEL LTDA EPP',
            'HOTEL TENDA LTDA',
            'HOTEL TERRANOVA',
            'HOTEL TOSCANO E RESTAURANTE',
            'HOTEL TROPICAL CAJURU LTDA ME',
            'HOTEL TURISMO PARQUE BALNEARIO LTDA',
            'HOTEL TURISMO PARQUE BALNEÁRIO',
            'HOTEL UNIVERSITARIO DE ITAPETININGA LTDA - ME',
            'HOTEL UNIVERSITARIO DE ITAPETININGA LTDA ME',
            'HOTEL URUBUPUNGA LTDA',
            'HOTEL VALE DO SOL LTDA-EPP',
            'HOTEL VALENTIM FLORIANOPOLIS LTDA',
            'HOTEL VALLE SUL LTDA',
            'HOTEL VARANDAS ARARAQUARA LTDA -ME',
            'HOTEL VAZ LOBO LTDA',
            'HOTEL VENTURA INN',
            'HOTEL VILA DO CONDE LTDA ME',
            'HOTEL VILA VERDE PENAPOLIS',
            'HOTEL VILA VERDE PENAPOLIS EIRELI',
            'HOTEL VILA VERDE PENAPOLIS EIRELI ME',
            'HOTEL VILA VERDE PENÁPOLIS EIRELE ME',
            'HOTEL VILLA BRITES.',
            'HOTEL VILLA DI RIMINI LTDA EPP',
            'HOTEL VILLA REBELLATO LTDA ME',
            'HOTEL VIVENDAS RIO CLARO LTDA',
            'HOTEL ÁGUA DAS ARARAS EIRELI',
            'HOTEL ÁGUA DAS ROSAS EIRELI',
            'HOTELARIA  ACCOR BRASIL S/A',
            'HOTELARIA ACCOR BARSIL S.A',
            'HOTELARIA ACCOR BRAASIL TDA',
            'HOTELARIA ACCOR BRASIL',
            'HOTELARIA ACCOR BRASIL  S/A',
            'HOTELARIA ACCOR BRASIL S.A',
            'HOTELARIA ACCOR BRASIL S.A.',
            'HOTELARIA ACCOR BRASIL S/A',
            'HOTELARIA ACCOR BRASIL S/A - SCPLE CORSIER FLAT SERVICE',
            'HOTELARIA ACCOR BRASIL SA',
            'HOTELARIA ACCOR BRASIL SA - SCP STELLA VEGA',
            'HOTELARIA ACCORINVEST BRASIL S.A',
            'HOTELARIA ACCORINVEST BRASIL S.A.',
            'HOTELARIA ACCORINVEST BRASIL S/A',
            'HOTELARIA ACCORINVEST BRASIL SA',
            'HOTELARIA ACCORRINVEST BRASIL S/A',
            'HOTELARIA AGISOL LTDA',
            'HOTELARIA AGISOL LTDA - EPP.',
            'HOTELARIA BRASIL LTDA',
            'HOTELARIA BRASIL LTDA.',
            'HOTELARIA CASTRO DE MARILIA LTDA ME',
            'HOTELARIA CASTRO DE MARÍLIA LTDA ME',
            'HOTELARIA ECO VALLE SFA LORENA LTDA',
            'HOTELARIA RIO S/C LTDA',
            'HOTELARIA WATERMANN LTDA ME',
            'HOTELEO HOTEIS E TURISMO LTDA',
            'HOTELS ADMINISTRAÇÃO HOTELEIRA LTDA',
            'HSP GONÇALVES HOTEL ME',
            'HTS EMPREENDIMENTOS HOTELEIROS LTDA- EPP',
            'HUESCA PALACE HOTEL LTDA EPP',
            'IBERICA S PRAIA HOTEL LIMITADA - EPP',
            'IBERICA S PRAIA HOTEL LTDA EPP',
            'IBIRAPUERA HOTEL & CONVENTION CENTER LTDA',
            'IBIRAPUERA HOTEL E CONVENTION CENTER LTDA',
            'IBIS BUDGET TAMBORÉ HOTELARIA ACCOR BRASIL S A',
            'IBIS HOTEL',
            'IBIS HOTELS',
            'IBIUNA TURISMO HOTEL EIRELI ME',
            'ICH ADMINISTRACAO DE HOTEIS S.A',
            'ICH ADMINISTRAÇÃO DE HOTEIS S.A',
            'ICH ADMINISTRAÇÃO DE HOTEIS S.A.',
            'ICH ADMINISTRAÇÃO DE HOTEIS SA',
            'IDEALI HOTEL LTDA - EPP',
            'IIB HOTELARIA E TURISMO LTDA',
            'IKEDA PROJETOS & HOTELARIA LTDA',
            'ILB HOTELARIA E TURISMO LTDA',
            'ILB HOTELARIA EIRELI -EPP',
            'ILHA PALACE HOTEL - ME',
            'ILHA PALACE HOTEL LTDA ME',
            'ILHA PORCHAT HOTEL LTDA',
            'ILHA PORCHAT HOTEL LTDA.',
            'ILHAFLET SERVIÇOS HOTELEIROS',
            'IMPERADOR PALACE HOTEL',
            'IMPERADOR PALACE HOTEL LTDA',
            'IND DE HOTEIS GUZZONI S.A.',
            'INDEPENDENCIA PLAZA HOTEL LTDA EPP',
            'INTERCITY ADMINISTRAÇÃO HOTELEIRA SE LTDA',
            'IPE CENTER HOTEL LTDA',
            'IPE PARK HOTEL LTDA',
            'IPÊ CENTER HOTEL LTDA',
            'IPÊ PARK HOTEL LTDA EPP',
            'IRMAOS PINA HOTEL E RESTAURANTE LTDA ME',
            'IRMAOS PINA HOTEL E RESTAURANTE LTDA-ME',
            'IRMÃOS PINA HOTEL E RESTAURANTE LTDA ME',
            'ITA PARK HOTEL LTDA - ME',
            'ITA PARK HOTEL LTDA ME',
            'ITAMARATI HOTEL E RESTAURANTE LTDA ME',
            'ITAPETINGA PLAZA HOTEL LTDA',
            'ITARARE HOTEL LIMITADA - ME',
            'ITARARE HOTEL LIMITADA ME',
            'ITARARÉ HOTEL ME',
            'ITU PLAZA HOTEL LTDA',
            'J C GOES HOTEL LTDA - ME',
            'J G HOTEIS E TURISMO LTDA',
            'J J HOTEIS E TURISMO LTDA',
            'J R PARK HOTEL',
            'J R PARK HOTEL LTDA EPP',
            'J. L. FERNANDES ADMINISTRAÇÃO HOTELARIA EIRELI',
            'J. P. PALACE HOTEL LTDA ME',
            'J.C.DE SÁ HOTEL - ME',
            'J.R. PARK HOTEL LTDA EPP',
            'JALES CENTER HOTEL LTDA',
            'JALES CENTER HOTELLTDA.-EPP',
            'JARUCHE HOTEIS LTDA',
            'JF DE GOUVEIA HOTEL ME',
            'JKLM HOTEL LTDA- EPP',
            'JOAO DE JESUS RODRIGUES BRAGANÇA PAULISTA HOTEL ME',
            'JOTARES HOTEIS E TURISMO LTDA',
            'JOÃO PAULINO DA SILVA HOTEL ME',
            'JOÃO RICARDO DONIZETE PELEGRINA ME BURKNER HOTEL',
            'JP PARK HOTEL LTDA EPP',
            'JR MARILIA HOTEIS EIRELI - EPP',
            'JR MARILIA HOTEIS EIRELI EPP',
            'JR MARILIA HOTEIS EIRELI-EPP',
            'JR MARILIA HOTEIS EIRELLI',
            'JR MARILIA HOTEIS LTDA EPP',
            'JR PARK HOTEL LTDA EPP',
            'JR PARK HOTEL LTDA.',
            'JR RIBEIRAO HOTEIS LTDA - EPP',
            'JR RIBEIRÃO HOTEIS LTDA EPP',
            'K. HOTEL LTDA - EPP',
            'KAFE HOTEL LTDA',
            'KATIA A. ALEXANDRE HOTEL-ME',
            'KK HOTEIS E TURISMO LTDA EPP',
            'KLINN HOTEL LTDA ME',
            'L. T. C. ADMINISTRAÇÃO DE HOTEIS LTDA EPP',
            'L.B. ADMINISTRAÇÃO DE HOTEIS LTDA-EPP',
            'L.F DESTEFANO HOTEL ME',
            'L.T.C. ADMINISTRAÇAO DE HOTEIS LTDA EPP',
            'LA HOTEIS EMPREENDIMENTOS 1 LTDA',
            'LA HOTELS EMPREENDIMENTOS 1 LTDA',
            'LA HOTELS EMPREENDIMENTOS 1 LTDA - (GT BRASILIA ALVORADA)',
            'LA HOTELS EMPRENDIMENTOS 1 LTDA',
            'LADY LAURA HOTEL LTDA. - ME',
            'LAF HOTEL LTDA - ME',
            'LAF HOTEL LTDA ME',
            'LAGOA SERENA FLAT HOTEL ADMINISTRADORA LTDA',
            'LAZARO ANGELO BALAN HOTEL ME',
            'LE PREMIER ADM DE HOTEIS E CONDOMINIOS LTDA',
            'LE PREMIER ADMINISTRADORA DE HOTÈIS E CONDOMÍNIOS LTDA',
            'LE PREMIER ADMINISTRATORA DE HOTEIS E CONDOMINIOS LTDAA',
            'LE PREVILEGE ADMINISTRADORA DE HOTEIS E CONDOMINIOS LTDA',
            'LE PREVILEGE ADMN DE HOTEIS E CONDOMINIOS LTDA',
            'LELIS HOTEL - BERENICE DO CARMO ALBANO FERIAN HOTEL - ME',
            'LEME´PLAZA HOTEL LTDA ME',
            'LIBANO EMPRESA PRESTADORA DE SERVICOS DE HOTELARIA EIREL',
            'LIDER HOTEL LTDA - EPP',
            'LIDIANE RODRIGUES PEREIRA HOTEL - ME',
            'LIETH LELLIS DE ASSIS CRUZ ME - HOTEL SÃO BENTO',
            'LILIAN CRISTINA FORTE LOPES HOTEL ME',
            'LIMA HOTEIS E TURISMO LTDA',
            'LITANI PARK HOTEL',
            'LITANI PARK HOTEL LTDA - EPP',
            'LITO PALACE HOTEL LTDA',
            'LIV HOTEL LTDA',
            'LIVIA CORREA LOPES EPP - OESTE PLAZA HOTEL',
            'LONDON HOTEL LTDA -ME',
            'LP HOTEIS LTDA',
            'LTC ADMINISTRAÇAO DE HOTEIS LTDA EPP',
            'M E DE OLIVEIRA DIAS HOTEL',
            'M F N DE LIMA EIRELI - GRAND HOTEL GUARUJA',
            'M. E. DE OLIVEIRA DIAS HOTEL',
            'M. E. DE OLIVEIRA DIAS HOTEL -ME',
            'M. E. DE OLIVEIRA DIAS HOTEL ME',
            'M.E. DE OLIVEIRA DIAS HOTEL',
            'M.E.DE OLIVEIRA DIAS HOTEL',
            'M.P.F. SERVICOS DE HOTELARIA LTDA-ME',
            'M.P.F. SERVIÇOS DE HOTELARIA LTDA',
            'MABRUK BARRETOS APART HOTEL LTDA',
            'MADUGA PALACE HOTEL LTDA.',
            'MAGA PLAZA HOTEL E TURISMO LTDA',
            'MANHATAN HOTEIS E TURISMO LTDA',
            'MANHATTAN HOTEIS E TURISMO LTDA',
            'MARCELO RICARDO DE SOUZA HOTELARIA',
            'MARCO MARCHETTI S/A HOTEIS',
            'MARDUY HOTEL LTDA',
            'MARIA PLAZA  HOTEL LTDA ME',
            'MARLENE APARECIDA DE OLIVEIRA BISPO HOTEL - ME',
            'MAURO COSTA DE OLIVEIRA HOTEL',
            'MAZZAROPI HOTELARIA LTDA',
            'MEILIA BRASIL 21 ADMINISTRAÇAO HOTELEIRA E COMERCIAL LTDA',
            'MELIA BRASIL ADMINISTRAÇÃO HOTELEIRA E COMERCIAL LTDA.',
            'MENDES HOTEIS TURISMO E ADMINISTRADORA LTDA',
            'MENDES HOTEIS TURISMO E ADMINISTRAÇÃO LTDA',
            'MENDES HOTEIS TURISMOS E ADMINISTRADORA LTDA.',
            'MEU HOTEL BOITUVA EIRELI',
            'MIRANTE PRAIA HOTEL',
            'MITSUKO HOTEL LIMITADA',
            'MONDRIAN SUITE HOTEL LTDA ME',
            'MONREALE HOTEL RIBEIRÃO PRETO',
            'MONTE SERRAT HOTEL LTDA-EPP',
            'MONTES & BROLEZE HOTEL E REFERIÇÃO LTDA',
            'MONTES E BROLEZE HOTEL E REFEICAO LTDA',
            'MPY HOTELARIA LTDA',
            'MUCHIUTT PARK HOTEL LTDA',
            'MULTIPLA ADM.DE HOTEIS EIRELI EPP',
            'MULTIPLA ADMI. DE HOTEIS EIRELLI EPP',
            'MULTIPLA ADMINISTRACAO DE HOTEIS LTDA',
            'MULTIPLA ADMINISTRAÇÃO DE HOTEIS EIRELI - EPP',
            'MULTIPLA ADMINISTRAÇÃO DE HOTEIS EIRELI-EPP',
            'MULTIPLA ADMINISTRAÇÃO DE HOTEIS EIRELLI - EPP.',
            'MULTIPLA ADMINISTRAÇÃO DE HOTÉIS EIRELI EPP',
            'N&A ADMINISTRAÇAO HOTELEIRA LTDA ME',
            'N&N HOTELARIA LTDA ME',
            'NAÇÕES PARK HOTEL LTDA',
            'NMOA POLLASTRINI HOTEL LTDA',
            'NOHOTEL AMERICANA LTDA',
            'NOHOTEL LTDA',
            'NOVO HORIZONTE HOTEIS E TURISMO LTDA',
            'NOVO HOTEL CARLTON SJCAMPOS LTDA',
            'NOVO HOTEL CARLTON SJCAMPOS LTDA EPP',
            'NOVO HOTEL CARLTON SJCAMPOS LTDA-EPP',
            'NOVO HOTEL PLAZA SJCAMPOS LTDA',
            'O MOINHO ITALIA HOTEL LTDA',
            'OASIS TOWER HOTEL LTDA',
            'OASIS TOWER HOTEL LTDA - ME',
            'OASIS TOWER HOTEL LTDA ME',
            'OBRAS SOCIAIS DA ARQUIDIOCESE DE APARECIDA HOTEL RAINHA DO BRASIL',
            'OESTE TURISMO E HOTELARIA LTDA - EPP',
            'OPTIMIM HOTEIS LTDA EPP',
            'ORG.HOTELEIRA ITAPETININGA LTDA',
            'ORGANIZAÇAO HOTELEIRA ITAPETININGA LTDA',
            'ORGANIZAÇÃO HOTELEIRA ITAPETININGA LTDA',
            'OROFINO HOTEL LTDA ME',
            'OURO HOTEL OURINHOS LTDA ME',
            'OURO HOTEL OURINHOS LTDA. - ME',
            'P & B HOTELARIA LTDA',
            'PAGURUS HOTELARIA EIRELI - EPP',
            'PALMARES HOTEL DE GUAIRA LTDA ME',
            'PANEGOSSI & PONTES HOTELARIA LTDA',
            'PANORAMA HOTEIS LTDA EPP',
            'PARADISE INN HOTEL CATANDUVA LTDA ME',
            'PARADISE INN HOTEL DE CATANDUVA LTDA ME',
            'PARAGEM HOTEL LTDA - EPP',
            'PARK TOWER HOTEL E CONVENCOES LTDA',
            'PASSER HOTEL LTDA',
            'PAULISTA NOVOESTE HOTEL E RESTAURANTE LTDA',
            'PAULISTA NOVOESTE HOTEL E RESTAURANTE LTDA ME',
            'PAULO OCTAVIO HOTEIS E TUR LTDA',
            'PAULO OCTAVIO HOTEIS E TUR. LTDA',
            'PAULO OCTAVIO HOTEIS E TUR. LTDA.',
            'PAULO OCTAVIO HOTEIS E TUR.LTDA',
            'PEKIN PALACE HOTEL LTDA',
            'PEKIN PALACE HOTEL LTDA.',
            'PELEGRINA E BRANDÃO LTDA GOLD INN HOTEL',
            'PERSONALITE ADMINISTRADORA DE HOTEIS E CONDOMINIOS LTDA',
            'PEVÊ-TUR HOTEIS LTDA',
            'PINDA PALACE HOTEL LTDA',
            'PIRAJUI PLAZA HOTEL LTDA - ME',
            'PIRAJUI PLAZA HOTEL LTDA ME',
            'PLAZA BRASILIA HOTEIS E TURISMO LTDA',
            'PLEASANT PLACE HOTEL LTDA EPP',
            'POLO HOTEL SJCAMPOS EIRELI',
            'POLO HOTEL SJCAMPOS LTDA',
            'PORTAL HOTEL MOGI MIRIM LTDA EPP',
            'PORTICO HOTEL E RESTAURANTE LTDA',
            'PORTO SEGURO HOTEL SALTO LTDA',
            'POUSADA HOTEIS LTDA',
            'POUSADA HOTEIS LTDA - EPP',
            'POUSADA HOTEIS LTDA EPP',
            'POUSADA HOTEIS LTDA.',
            'PREMIUM ADMINISTRAÇAO HOTELEIRA LTDA',
            'PRIMAR PLAZA HOTEL LDTA EPP',
            'PRIMAR PLAZA HOTEL LTDA EPP',
            'PRISMA PLAZA HOTEL',
            'PROJETA HOTEL E EMPREENDIMENTOS IMOBILIARIOS LTDA',
            'PROJETA HOTEL E EMPREENDIMENTOS IMOBILIÁRIOS LTDA',
            'QUILIMANJARO COMERCIO DE ALIMENTOS E HOTELARIA LTDA',
            'R DE CARVALHO HOTEL LTDA ME',
            'R MASSON HOTEIS LTDA',
            'R MASSON HOTEIS LTDA  ME',
            'R. MASSONI HOTEIS LTDA - ME',
            'RANCHO DAS PEDRAS HOTEL LTDA',
            'RB3 HOTELARIA LTDA',
            'RCA SERV DE HOTELARIA LTDA EPP',
            'REAL GARDEN HOTEL LTDA - ME',
            'REAL GARDEN HOTEL LTDA ME',
            'REAL HOTEL SUD MENNUCCI LTDA.-ME',
            'REAL PALACE HOTEL BEBEDOURO LTDA',
            'REAL PALACE HOTEL PREMIUM GOLD LTDA-ME',
            'REALCE HOTEL & EVENTOS LTDA',
            'REDE HOTELEIRA BAOBA LTDA',
            'REGIONAL PLAZA HOTEL',
            'REGIONAL PLAZA HOTEL & EVENTOS',
            'REGIONAL PLAZA HOTEL & EVENTOS OLICIO BOCCHI',
            'RENATA BONVENTI MACHADO HOTEL LTDA',
            'RENATO A. BUSQUETE SERVIÇOS DE HOTELARIA - ME',
            'REQUINTS HOTEL LTDA-ME',
            'RESIDENCIAL HOTEL BOTUCATU LTDA EPP',
            'RESTAURANTE HOTEL COUNTRY',
            'RIBEIRO & ROCHA HOTEL LTDA',
            'RIELLIS CENTER HOTEL LTDA',
            'RIO PRETO PARTNERS HOTEIS LTDA',
            'RIVIBAU HOTEIS E TURISMO LTDA',
            'RIVIBAU HOTEIS E TURISMO LTDA.',
            'ROBUSTI PLAZZA HOTEL',
            'ROCHA PALACE HOTEL LTDA,.-ME',
            'ROHEDAMA HOTEL E LANCHONETE LTDA',
            'S.G.P. HOTEL LTDA',
            'SA HOTEL LTDA',
            'SAFE ADMINISTRAÇAO HOTELARIA LTDA',
            'SAFE ADMINISTRAÇÃO HOTELARIA CAPÃO BONITO LTDA',
            'SAFE ADMINISTRAÇÃO HOTELEIRA LTDA.',
            'SAKR HOTEL LTDA',
            'SAKR HOTEL RIO PRETO LTDA EPP',
            'SAN FRANCISCO PARK HOTEL EIRELI',
            'SAN JUAN PALACE HOTEL',
            'SAN JUAN PALACE HOTEL - EIRELI ME',
            'SAN JUAN PALACE HOTEL - EIRELLI - ME',
            'SAN JUAN PALACE HOTEL EIRELI ME',
            'SAN MARCO HOTEL',
            'SAN MICHEL PALACE HOTEL',
            'SAN MICHEL PALACE HOTEL LTDA',
            'SAN MICHEL PALACE HOTEL LTDA - EPP',
            'SANBRU - HOTEL E RESTAURANTE LTDA ME',
            'SANDRA H. GRITTI MARQUES E CIA LTDA EPP - ONIX HOTEL',
            'SANTA FELICIDADE HOTEL EVENTOS LTDA - ME',
            'SANTOS & CAPEL HOTEL LTDA',
            'SANTOS E CITADINI SERVIÇOS DE HOTELARIA E AGROPECUARIA LTDA',
            'SANTUÁRIO PALACE HOTEL LTDA',
            'SAO ROQUE PARK HOTEL',
            'SAO ROQUE PARK HOTEL LTDA ME',
            'SERGIO LUIS ROLA - HOTEL ME',
            'SERRA NEGRA PALADIUM HOTEL LTDA',
            'SHAFIRA HOTEL LTDA',
            'SIND. DOS EMP. NO COM. HOTELEIRO E SIMILARES SP.',
            'SLAVIERO EMPREENDIMENTOS HOTELEIROS LTDA',
            'SMART ADMINISTRADORA HOTELEIRA LTDA',
            'SOCIEDADE HOTEL PALACE LTDA',
            'SOCIEDADE HOTEL PALACE LTDA EPP',
            'SOLAR DE PEDRA HOTEL',
            'SONOTEL HOTEL LTDA',
            'SORAYA CAHIME - HOTEL - EPP',
            'SOTAM HOTEL LTDA',
            'SPAN HOTELARIA LTDA',
            'ST PAUL PLAZA HOTEIS E TURISMO LTDA',
            'ST. PAUL PLAZA HOTEIS E TURISMO LTDA',
            'SUELI T. E. ERAS ADMINISTRAÇÃO HOTELEIRA - ME',
            'SUMMIT HOTELARIA LTDA',
            'SÃO ROQUE PARK HOTEL LTDA-ME',
            'TANIA MARIA CERNIAUKAS HOTELARIA',
            'TANIA MARIA CERNIAUKAS HOTELARIA ME',
            'TANIA MARIA CERNIAUKAS HOTELARIA-ME',
            'TAU FLATS EMPREENDIMENTOS DE HOTELARIA E PARTICIPAÇÕES LTDA',
            'TECNOFLAT GESTAO E HOTELARIA S/C LTDA',
            'TEIXEIRA & VIZENTINI HOTEL LTDA ME',
            'TERRAÇO PEREA HOTEL LTDA - ME',
            'THE HILL EMPREENDIMENTO HOTELEIROS LTDA',
            'THERMAS PARK HOTEL',
            'TIFFANY HOTEL LTDA',
            'TIFFANY HOTEL LTDA.',
            'TORRE HOTEL EIRELI',
            'TOWER FRANCA HOTEL',
            'TOWER FRANCA HOTEL LTDA',
            'TRAJANO PALACE HOTEL LTDA',
            'TREVIS & BRAGATO HOTEL LTDA',
            'TREVISI & BRAGATO HOTEL LTDA',
            'TUTU HOTEL LTDA ME',
            'UBATUBA PRAIA GRANDE HOTEL LTDA',
            'UNIQUE SERV HOT E ALIM S/A',
            'UNIQUE SERV.DE HOTELARIA ALIM.COM PART.S/A',
            'V.E.C.HOTEIS LTDA',
            'VALE RESIDENCE HOTEL LTDA - ME',
            'VALE SERRA MAR HOTEL E SERVIÇOS LTDA',
            'VALE VERDE PALACE HOTEL GARDELLI & BATTAGLINI LTDA EPP',
            'VALENTIM PALACE HOTEL',
            'VICTORIA PARK HOTEL HOTELARIA E TURISMO LTDA',
            'VICTORIA PARK HOTEL HOTELARIA E TURISMO LTDA.',
            'VILLA DE HOLANDA HOTEL LTDA',
            'VILLA MAIOR HOTEL LTDA - ME',
            'VILLAGE PALACE HOTEL LTDA ME',
            'VILLAS BOAS HOTEL LTDA - ME',
            'VILLE ATLANTICO HOTEIS E TURISMO LTDA',
            'VILLE HOTEL GRAMADAO',
            'VILLE HOTEL GRAMADAO DE VOTUPORANGA LTDA',
            'VILLE HOTEL GRAMADAO DE VOTUPORANGA LTDA ME',
            'VILLE HOTEL GRAMADAO DE VOTUPORANGA LTDA-ME',
            'VILLE HOTEL GRAMADO DE VOTUPORANGA LTDA',
            'VILLE PARK HOTEL LTDA',
            'VILLE PARK HOTEL LTDA - ME',
            'VILLE PARK HOTEL LTDA ME',
            'VILLE PARK HOTEL ME',
            'VITORIA HOTEIS LTDA',
            'VITORIA HOTEL CAMPINAS LTDA',
            'VITORIA REGIA HOTEL LIMITADA',
            'VITORIA REGIA HOTEL LTDA',
            'VIVENDA HOTEL LTDA',
            'W-TUR HOTEIS E SIMILARES LTDA',
            'WANNA HOTEL LTDA - ME',
            'WINDSOR ADMINISTRAÇÃO DE HOTÉIS E SERVIÇOS LTDA'], 
            'hospedagem', inplace=True)

In [84]:
fthotel = df.fornecedor == 'hospedagem'
dfhotel = df.loc[fthotel]
dfhotel

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
1909,2022,300098,2,196.00,1.061991e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
1910,2022,300098,2,225.63,3.471411e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
1911,2022,300098,2,248.17,9.632945e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
1934,2022,300098,3,315.00,3.471411e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
1936,2022,300098,3,468.00,9.632945e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
...,...,...,...,...,...,...,...
118336,2022,300655,7,1080.00,5.211653e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
118337,2022,300655,7,1638.00,2.919185e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
118338,2022,300655,7,1212.00,3.430618e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
118369,2022,300655,8,1085.76,5.949818e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem


### Filtro A

In [85]:
# criando um filtro para tipo A
filtroa = df.tipo == 'A - COMBUSTÍVEIS E LUBRIFICANTES'

In [86]:
# cria um df filtrado com as informações da categoria A
dcombustivel = df.loc[filtroa]

In [87]:
dcombustivel 

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
0,2013,300098,1,575.53,5.049700e+11,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO VANUTRE LTDA
1,2013,300098,1,1650.76,4.881450e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,S.D.LUIZARI & CIA. LTDA.
2,2013,300098,1,1799.29,6.586291e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO EXECUTIVO LTDA
21,2013,300098,2,602.34,5.049700e+11,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO VANUTRE LTDA
22,2013,300098,2,20.00,1.462152e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
...,...,...,...,...,...,...,...
118394,2022,300655,9,161.85,4.751503e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,POSTO BRIGADEIRO LTDA
118395,2022,300655,9,189.00,5.100008e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO MICHELE LTDA
118396,2022,300655,9,126.00,5.928040e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,POSTO DE GASOLINA SAO FRANCISCO LT
118397,2022,300655,9,206.77,6.192941e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,POSTO PRESIDENTE J K LT


In [88]:
# cria um filtro para pegar partes de string que podem estar nos dados relacionados a combustivel
filtrocomb = dcombustivel.fornecedor.str.contains('COMB')

In [89]:
# cria um novo df com base no df filtrado, passando mais um filtro de parametro
dfcombustivel = dcombustivel.loc[filtrocomb]

In [90]:
dfcombustivel

,ano,matricula,mes,valor,cnpj,tipo,fornecedor


In [91]:
# encontra dos nomes de fornecedor que querem dizer a mesma coisa
sorted(pd.unique(dfcombustivel['fornecedor']))

[]

In [92]:
# compatibilizando dados de fornecedor para eles terem o mesmo nomes
df.replace(['2 AMIGOS COM. DE COMBUST. DE MONTE APRAZIVEL LTDA.',
            '2 AMIGOS COMERCIO DE COMBUSTIVEIS LTDA',
            'A T GARCIA COMBUSTIVEIS',
            'A Z COMERCIO DE COMBUSTIVEIS LTDA',
            'A Z COMERCIO DE COMBUSTÍVEIS LTDA.',
            'A Z COMÉRCIO DE COMBUSTÍVEIS LTDA.',
            'ABDELNOR COMERCIO DE COMBUSTIVEIS LTDA',
            'ABDELNOR II COM DE COMBUSTIVEIS LTDA',
            'ABDELNOR III COMERCIO COMBUSTIVEIS LTDA',
            'ABDELNOR III COMERCIO DE COMBUSTIVEIS LTDA',
            'ABDELNOR VI COMERCIO DE COMBUSTIVEIS LTDA',
            'ABUDABI COMERCIO COMBUSTIVEL LTDA',
            'ADG COMERCIO DE COMBUSTIVEIS LTDA',
            'AEROCAR COMÉRCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'AGILE COMBUSTIVEIS E CONVENIENCIA LTDA',
            'AGROCOMB COMBUSTIVEIS LTDA EPP',
            'ALCATRAZ COMBUSTIVEIS E CONVENIENCIA LTDA',
            'ALCATRAZ COMBUSTÍVEIS E CONVENIÊNCIA LTDA',
            'ALEXANDRE ROBERTI COMBUSTIVEIS EPP',
            'ALTERNATIVA COM DE COMBUSTIVEIS LTDA',
            'ALTERNATIVA COMERCIO DE COMBUSTÍVEIS LTDA EPP',
            'ALVARENGA COMERCIO VAREJISTA DE COMBUSTIVEL LTDA',
            'ALVES D ABREU E SALGADO COMB. COM. EE',
            'ALVES D ABREU E SALGADO COMBUSTÍVEIS COMÉRCIO E E',
            'ALVES D ABREU E SALGADO-COMBUS.COM.E ESTAC.LTDA',
            'ALVES D ABREU E SALGADO-COMBUST COM E ESTAC LTDA',
            'AMERICA CEN DE ABAS DE COMB LTDA',
            'AMERICA CENTRO DE ABAST DE COMBUSTÍVEIS LTDA',
            'AMERICA CENTRO DE ABASTECIMENTO DE COMBUSTIVEIS',
            'AMERICA CENTRO DE ABASTECIMENTO DE COMBUSTIVEIS LTDA',
            'AMERICAMANIA COM. VAREJISTA DE COMBUSTIVEIS LTDA',
            'AMERICAMANIA COMÉRCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'AMERICANA COMERCIO VAREJISTA DE COMBUSTIVEL L',
            'AMF AEROPORTO COM VAREJISTA DE COMBUSTÍVEIS LTDA',
            'AMF AEROPORTO COMERCIO VAREJISTA DE COMBUST LTDA',
            'AMF AEROPORTO COMERCIO VAREJISTA DE COMBUSTIVEL LTDA',
            'AMP COMBUSTIVEIS MAGNOLIA LTDA',
            'ANDRADE BEBEDOURO COMERCIO DE COMBUSTIVEIS LTDA',
            'ANTONIO BISPO-COMBUSTIVEIS.',
            'AQUARIUS COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'ARIELLA IMIGRANTES COMERCIO VAREJISTA DE COMBUST',
            'ARINELA IMIGRANTES COMERCIO VAREJISTA COMBUSTIVEL',
            'ARINELLA BANDEIRANTES COMERCIO VAREJISTA DE COMBU',
            'ARINELLA BANDEIRANTES COMERCIO VAREJISTA DE COMBUSTIVEIS',
            'ARINELLA BANDEIRANTES COMERCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'ARINELLA IMIGRANTES COM. VAR.COMBUST. LTDA',
            'ARINELLA IMIGRANTES COM.VAR.DE COMB.LTDA',
            'ARINELLA IMIGRANTES COMERCIO VAREJISTA DE COMBUST',
            'ARINELLA IMIGRANTES COMERCIO VAREJISTA DE COMBUSTIVEL.',
            'ARINELLA IMIGRANTES COMÉRCIO VAREJISTA DE COMBUSTÍVEL',
            'ARTAM ARARAQUARA COMERCIO DE COMBUSTIVEIS LTDA',
            'ARTAM ARARAQUARA COMÉRCIO DE COMBUSTÍVEIS LTDA',
            'ARTAM ARARAQUARA DE COMBUSTIVEL LTDA',
            'ARTAM COMERCIO DE COMBUSTIVEIS LTDA',
            'ARTAM SAO CARLOS COMERCIO DE COMBUSTIVEIS LTDA',
            'ARTAM SAO CARLOS COMERCIO DE COMBUSTIVEIS LTDA.',
            'ASTR0 SUL POSTO DE COMBUSTIVEIS LTDA',
            'ASTRO COMERCIO DE COMBUSTIVEIS  LTDA',
            'ASTRO COMERCIO DE COMBUSTIVEIS LTDA',
            'ASTRO COMERCIO DE COMBUSTÍVEIS LTDA',
            'ASTRO COMÉRCIO DE COMBUSTIVEIS  LTDA',
            'ASTRO SUL POSTO DE COMBUSTIVEIS LTDA',
            'ASTRO SUL POSTO DE COMBUSTIVEL LTDA',
            'ATLANTA ROTA SHOPPING COMERCIO DE COMBUSTÍVEIS EIRELI',
            'AUTO POSTO ALMIRANTE RANGEL COMÉRCIO DE COMBUSTÍVEIS E DERIVADOS DE PETRÓLEO',
            'AUTO POSTO BARAO SUPERVE COMBUSTIVEIS LTDA',
            'AUTO POSTO DE COMB. DE SUZ. LTDA',
            'AUTO POSTO DE COMBUSTÍVEIS V2 LTDA',
            'AUTO POSTO DELEVATTI COMERCIO DE COMBUSTIVEIS LTDA',
            'AUTO POSTO E SERVIÇO CENTRAL COMBUSTIVEIS LTDA',
            'AUTO POSTO EMIRADOS COMBUSTÍVEIS',
            'AUTO POSTO J S A COMBUSTIVEIS LTDA',
            'AUTO POSTO LUPUS COMBUSTÍVEIS LTDA',
            'AUTO POSTO MALHEIROS COM. DE COMB. LTDA.',
            'AUTO POSTO MALHEIROS COMERCIO DE COMBUSTÍVEIS LTDA',
            'AUTO POSTO NOVO HORIZONTE II - ALMEIDA E OLIVEIRA COMERCIO DE COMBUSTIVEIS E DERIVADOS DE P',
            'AUTO POSTO SUZUKI COMÉRCIO DE COMBUSTÍVEIS LTDA',
            'AUTO POSTO SÃO SEBASTIÃO COMÉRCIO DE COMBUSTÍVEIS',
            'AUTO POSTO TATUIMAR LTDA - POSTO COMBOIO',
            'AUTO POSTO VILA FLORIANO XAVIER E XAVIER COMERCIO DE COMBUSTIVEIS E LUBRIF',
            'AUTO POSTO VILA NOVA COMERCIO DE COMBUSTIVEIS EIRELI',
            'AVIGNON COM. VAREJISTA DE COMBUSTIVEIS LTDA',
            'AZ COMERCIO DE COMBUSTIVEIS LTDA',
            'B J COM DE COMB E LUBR DE OURINHOS LTDA',
            'B J COM DE COMBUSTIVEIS E LUBR DE OURINHOS LTDA',
            'B2F COM. DE COMBUSTÍVEL LTDA',
            'B2F COMERCIO DE COMBUSTIVEL',
            'BASSO E ADANI COMERCIO DE COMBUSTIVEL E TRANSPORTE',
            'BEIRA MINHO COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'BEIRA MINHO COMERCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'BEIRA MINHO COMERCIO VERJISTA DE COMBUSTIVEIS',
            'BELA VISTA COM VAREJO DE COMBUSTIVEIS LTDA ME',
            'BELA VISTA COMERCIO VAREJISTA DE COMBUSTÍVEIS LTD',
            'BELA VISTA COMERCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'BERNARDO & ROSSINI COM. DE COMBUSTIVEIS LTDA',
            'BERNARDO E ROSSINI COMERCIO DE COMBUSTIVEIS LTDA',
            'BIRI PETRO COM. DE COMBUSTIVEIS LTDA',
            'BLV COMÉRCIO DE COMBUSTIVEIS LTDA',
            'BLW COMERCIO DE COMBUSTÍVEIS LTDA',
            'BLW COMÉRCIO DE COMBUSTÍVEIS LTDA.',
            'BOITUVA COM. DE COMBUSTIVEIS LTDA',
            'BOITUVA COMERCIO DE COMBUSTIVEIS LTDA',
            'BOITUVA COMÉRCIO DE COMBUSTÍVEIS LTDA',
            'BOM JESUS AUTO POSTO 3F COMERCIO DE COMBUSTIVEIS LTDA',
            'BORELI CENTER PETRO COM. DE COMBUSTIVEIS EIRELI',
            'CABRAL§LOCATELLI COM DE COMBUSTIVEIS LTDA',
            'CAETES COMERCIO DE COMBUSTIVEIS LTDA',
            'CALOPSYTA COMERCIO DE COMBUSTIVEIS E DERIVADOS DE',
            'CALUANA COMBUSTIVEIS E SERVICOS LTDA',
            'CALUANA COMBUSTIVEIS E SERVIÇOS LTDA',
            'CALUANA COMBUSTÍVEIS E SERVIÇOS',
            'CANABRAVA III COM DE COMB DE GARÇA LTDA',
            'CARMEL COM. DE COMBUSTÍVEIS E SERVIÇOS LTDA',
            'CARONI COMERCIO DE COMBUSTIVEIS LTDA',
            'CASA AVENIDA COM DE COMBUSTIVEL LTDA',
            'CASA AVENIDA COM. DE COMBUST. LTDA',
            'CASA AVENIDA COMERCIO DE COMBUSTIVEIS LTDA',
            'CASA AVENIDA COMERCIO DE COMBUSTIVEL LTDA',
            'CASSIANO COMERCIO DE COMBUSTÍVEIS E CONVENIENCIAS',
            'CENTRO ALTO CONVENIENCIA E COMBUSTIVEIS LTDA',
            'CENTRO AUTOMOTIVO DE COMBUSTIVEIS AMERICANA',
            'CENTRO AUTOMOTIVO DE COMBUSTIVEIS AMERICANA LTDA',
            'CENTRO AUTOMOTIVO E COM DE COMBUSTIVEIS MAXX',
            'CENTRO AUTOMOTIVO E COMERCIO DE COMBUSTIVEIS MAXX',
            'CENTRO AUTOMOTIVO E COMERCIO DE COMBUSTÍVEIS MAXX',
            'CENTRO AUTOMOTIVO E COMÉRCIO DE COMBUSTIVEIS MAXX',
            'CENTRO COMERCIAL DE COMBUSTIVEIS HORTOLANDIA LTDA',
            'CENTRO DE COMBUSTIVEIS E SERVIÇOS VENEZA LTDA',
            'CHACARA DO QUIRIRI COM VAREJ DE COMBUSTIVEL',
            'CHACARA DO QUIRIRI COMERCIO VAREJISTA DE COMBUSTI',
            'CHAPARRAL COMERCIO DE COMBUSTIVEIS AUTOMOTIVOS LT',
            'CHIESA PARQUE INDUSTRIAL COMBUSTIVEIS LTDA',
            'CHORAO COMBUSTIVEIS LTDA',
            'CHORORAO COMBUSTIVEIS LTDA',
            'CIDADE DO VINHO COM. VAREJ. DE COMB. LTDA',
            'CINADIS REVENDEDORA DE COMBUSTIVEIS LTDA',
            'CINADIS REVENDEDORA DE COMBUSTÍVEIS LTDA',
            'CINTRA COM COMBUSTÍVEIS LTDA',
            'COLISEU COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'COLUMBUS COMERCIO DE COMBUSTIVEIS LTDA',
            'COM DE COMB BRAUNA LTDA',
            'COM DE COMB. OLIVEIRA E RAMOS LTDA EPP',
            'COM.VAR. DE COMB. LUMAR TAIPAS EIRELI',
            'COMBOIO POSTO DE SERVIÇO LTDA',
            'COMBOY AUTO POSTO LTDA',
            'COMERCIAL DE COMBUSTISTVEIS APOLLO PIRACICABA LTDA',
            'COMERCIAL DE COMBUSTIVEIS ALPHAPOLLO LTDA',
            'COMERCIAL DE COMBUSTIVEIS BOA VISTA LTDA',
            'COMERCIAL DE COMBUSTIVEIS OUROESTE',
            'COMERCIAL DE COMBUSTIVEIS PENA VERDE LTDA',
            'COMERCIAL DE COMBUSTIVEIS VALPETRO LTDA',
            'COMERCIAL DE COMBUSTIVEIS VISCONDE US LTDA',
            'COMERCIAL DE COMBUSTIVEL APOLLO RIO CLARO LTDA',
            'COMERCIAL DE COMBUSTÍVEIS GUARANI US LTDA ME',
            'COMERCIAL DE COMBUSTÍVEIS GUARANI US LTDA.',
            'COMERCIAL NOGUEIRENSE DE COMBUSTIVEL LTDA',
            'COMERCIAL NOROESTE DE COMBUSTIVEIS LTDA',
            'COMERCIAL R B COMBUSTIVEIS LTDA',
            'COMERCIAL R B R DE COMBUSTÍVEIS LTDA',
            'COMERCIAL RBR DE COMBUSTIVEIS LTDA',
            'COMERCIO DE COMBUSTIVEIS AFC LTDA',
            'COMERCIO DE COMBUSTIVEIS E TRANSP BORTOLINI LTDA',
            'COMERCIO DE COMBUSTIVEIS GENESIS LTDA',
            'COMERCIO DE COMBUSTIVEIS MIRANTE LTDA',
            'COMERCIO DE COMBUSTIVEIS MORAES SALES LTDA',
            'COMERCIO DE COMBUSTIVEIS PASTORELLO LTDA.',
            'COMERCIO DE COMBUSTIVEIS RUY RODRIGUES LTDA',
            'COMERCIO DE COMBUSTIVEIS SANTO EXPEDITO LTDA',
            'COMERCIO DE COMBUSTIVEIS SAO FRANCISCO LTDA',
            'COMERCIO DE COMBUSTIVEIS VALE DO R. PRETO LTDA-EPP',
            'COMERCIO DE COMBUSTIVEL GRANADO LTDA',
            'COMERCIO DE COMBUSTIVEL RUY RODRIGUES KTDA',
            'COMERCIO DE COMBUSTÍVEIS MORAES SALES LTDA',
            'COMERCIO DE COMBUSTÍVEIS SANTO EXPEDITO LTDA',
            'COMERCIO VAREGISTA DE COMBUSTIVEIS PINHO DO GRAJA',
            'COMERCIO VAREGISTA DE COMBUSTIVEIS RT DEZOITO E M',
            'COMERCIO VAREJISTA DE COMBUSTIVEIS FOLENA CURVAO',
            'COMERCIO VAREJISTA DE COMBUSTIVEIS JARDIM RIVIERA LTDA',
            'COMERCIO VAREJISTA DE COMBUSTIVEIS PINHO DO GRAJAU',
            'COMERCIO VAREJISTA DE COMBUSTIVEIS RT DEZOITO E M',
            'COMERCIO VAREJISTA DE COMBUSTIVEIS VIA SUL LTDA',
            'COMERCIO VAREJISTA DE COMBUSTÍVEIS FIG LTDA',
            'COMERCIO VAREJISTA DE COMBUSTÍVEIS PINHO DO GRAJA',
            'COMERCIOVAREJISTA DE COMBUSTIVEIS PARAISO DE BARUERI LTDA',
            'COML PHILADELPHIA DE COMBUSTIVEIS LT',
            'COML PHILADELPHIA DE COMBUSTÍVEIS LT',
            'COMÉRCIO DE COMBUSTIVEIS E TRANSPORTES BORTOLINI LTDA.',
            'COMÉRCIO DE COMBUSTÍVEIS BONFIM LTDA.',
            'CONCORDIA POSTO DE COMBUST E LUBRIFICANTES',
            'CONQUISTA DE MAIRIPORA COMERCIO VAREGISTA DE COMBUSTIVEL',
            'CONQUISTA DE MAIRIPORA COMERCIO VAREJISTA DE COMB',
            'COSTA & COSTA COM DE COMB. LTDA',
            'COSTA & COSTA COM DE COMBUSTÃ\x8dVEIS LTDA',
            'COSTA E JORDAO COMBUSTIVEIS LTDA',
            'CUESTA PETRO COMERCIO DE COMBUSTIVEIS LTDA',
            'CV COMERCIO DE COMBUSTIVEIS',
            'CV COMERCIO DE COMBUSTÍVEIS LTDA',
            'DEBORA COM COMBUST DERIVADOS PETROLEO LTDA',
            'DEBORA COMERCIO DE COMBUSTIVEIS E DERI DE PETROLEO LTDA',
            'DEL REY COMBUSTIVEIS LTDA',
            'DELEVATTI COMERCIO DE COMBUSTIVEIS LTDA',
            'DELEVATTI COMERCIO DE COMBUSTIVEIS NOVO HORIZONTE',
            'E A TUSCHI COMBUSTIVEIS',
            'E C E ZANELLA COMERCIO DE COMBUSTIVEIS LTDA',
            'E. GARCIA COMERCIO DE COMBUSTIVEIS LTDA.',
            'ECO KLABIN COMÉRCIO VAREJISTA DE COMBUSTÍVEIS LTDA.',
            'ECO POSTO DE COMBUSTIVEIS JARDIM LTDA',
            'ECO POSTO W F COMBUSTIVEL E RESTAURANTE LTDA',
            'ECO POSTO W F COMBUSTÍVEL E RESTAURANTE LTDA',
            'ECO POSTO W. F. COMBUSTÍVEIS E RESTAURANTE LTDA.',
            'ECONOPETRO COMBUSTÍVEIS E SERVIÇOS EIRELI',
            'ECRX COMERCIO DE COMBUSTIVEIS',
            'EDIVALDO FERNADO CABRAL COMBUSTIVEIS LTDA',
            'EDUCANDARIO COM DE COMBUSTIVEIS LTDA',
            'EDUCANDARIO COMERCIO DE COMBUSTIVEIS LTDA',
            'EJ COMERCIO DE COMBUSTIVEIS EIRELI',
            'ESPACIAL COMERCIO DE COMBUSTIVEIS LTDA',
            'ESPIGAO COMERCIO DE COMBUSTIVEIS LTDA',
            'ESPIGÃO COMERCIO DE COMB. LTDA',
            'ESPIGÃO COMERCIO DE COMBUSTÍVEL LTDA',
            'EUROPA COMERCIO DE COMBUSTIVEIS LTDA',
            'EXPRESS COMERCIO DE COMBUSTIVES E LUBRIFICANTES',
            'F G NOVO CENTRO POSTO DE COMBUSTIVEIS LTDA',
            'F J R COMERCIO DE COMBUSTIVEIS LTDA',
            'FARIA LIMA COMERCIO DE COMBUSTÍVEL LTDA',
            'FAROL DA AGUA FRIA COMERCIO VAREJISTA DE COMBUSTÍVEL',
            'FASE QUATRO COMERCIO DE COMBUSTIVEIS LTDA',
            'FASE QUATTRO COMERCIO DE COMBUSTIVEIS LTDA',
            'FG NOVO CENTRO POSTO DE COMBUSTIVEIS',
            'FIBRA POSTO DE COMBUSTIVEIS LTDA',
            'FLMA DIADEMA COMERCIO VAREJISTA DE COMBUSTIVEIS',
            'FMJA COMERCIO DE COMB.LTDA',
            'FMJA COMERCIO DE COMBUSTIVEIS LTDA',
            'FORMIGAO COMERCIO DE COMB.LTDA',
            'FORMIGAO COMERCIO DE COMBUSTIVEIS E LUBRIFICANTES',
            'FORMIGÃO COM. DE COMBUST. E LUBRIFICANTES - LTDA',
            'FOX MILENIUM COMERCIO DE COMBUSTIVEIS LTDA',
            'FRANPETRO COMERCIO DE COMBUSTIVEIS LTDA',
            'FRASCHETTI COMERCIO DE COMBUSTIVEIS LTDA',
            'FT COMERCIO DE COMBUSTIVEIS LTDA',
            'G A COMERCIO DE COMBUSTIVEIS LTDA',
            'G M ABBADE COMBUSTIVEIS',
            'G S COMERCIO DE COMBUSTIVEIS LTDA',
            'GABRIELLA GAS STATION COM DE COMBUSTIVEIS LTDA',
            'GASPER COMBUSTIVEIS LTDA',
            'GAT POSTO DE COMBUSTIVEIS LTDA',
            'GAT POSTO DE COMBUSTÍVEIS LTDA',
            'GENERAL COMERCIO DE COMBUSTIVEIS LTDA',
            'GENERAL COMERCIO DE COMBUSTIVEL LTDA',
            'GF COMERCIO VAREJISTA DE COMBUSTIVEL LTDA',
            'GRAMINHA COMERCIO DE COMBUSTIVEIS LTDA',
            'GRAMINHA COMERCIO DE COMBUSTÍVEIS',
            'GRANADA POSTO DE COMBUSTIVEIS LTDA',
            'GS POSTO DE COMBUSTIVEL LTDA',
            'GUAIRA COMERCIO DE COMBUSTIVEIS LTDA',
            'IGNOTTI COM COMBUST E LUBRIFICANTES LT',
            'IGNOTTI COM COMBUSTIVEIS E LUBRIFICANTES LTDA',
            'IGNOTTI COM. COMBUST. E LUBRIFICANTES - LTDA',
            'IGNOTTI COM. COMBUSTIVEIS',
            'IGO COM. VAREJISTA DE COMBUSTÍVEL',
            'IGO COMÉRCIO VAREJISTA DE COMBUSTÍVEIS',
            'IGO-COMERCIO VAREJISTA DE COMBUSTIVEL LTDA',
            'IMIGRANTE COMERCIO DE COMBUSTIVEIS',
            'IMIGRANTES COM. DE COMB. E DER. DE PETRÓLEO LTDA',
            'IMIGRANTES COMÉRCIO DE COMB. E DERIV.PETR.LTDA.',
            'IPE COMERCIO COMBUSTIVEIS LTDA',
            'IRMAOS GUERRERO POSTOS DE COMBUSTIVEIS LTDA',
            'ITAPIRENSE COMBUSTÍVEIS LTDA.',
            'J L C COMERCIO DE COMBUSTIVEIS E DERIVADOS DE PET',
            'J M COMERCIO DE COMBUSTIVEIS LTDA',
            'J P LEAL COMBUSTÍVEIS E POSTOS DE SERVIÇOS LTDA',
            'J S COMERCIO DE COMBUSTIVEIS LTDA',
            'JAPÃO COMERCIO DE COMBUSTIVEIS LTDA',
            'JJ PETRO COMBUSTIVEIS LTDA',
            'JLC COMERCIO DE COMBUSTIVEL E DERIVADOS DE PET',
            'JN COMERCIO DE COMBUSTIVEIS E LUBRIFICANTES LTDA',
            'JN COMERCIO DE COMBUSTÍVEIS E LUBRIFICANTE',
            'JN COMERCIO DE COMBUSTÍVEIS E LUBRIFICANTES',
            'JOAO MANTOVANI COMBUSTIVEIS EPP',
            'JOAO ROBERTO RODRIGUES COMBUSTIVEIS',
            'JORGE S ASADA COMB E TRANSP EPP',
            'JUVENAL PARADA COM.VAREJ.DE COMB.LTDA',
            'JUVENAL PARADA COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'JUVENTOS COMÉRCIO VAREGISTA DE COMBUSTIVEIS LTDA',
            'KRAFTECK COMERCIO DE COMBUSTIVEL',
            'L & M MATOS COMBUSTIVEIS LTDA',
            'L E M MATOS COMBUSTIVEIS',
            'L E M MATOS COMBUSTIVEIS LTDA',
            'L E M MATOS COMBÚSTIVEIS LTDA',
            'L F J COMERCIODE COMBUSTIVEL LTDA',
            'L.BERTANHA COMBUSTIVEIS EPP',
            'LEANDRINI AL POSTO COMBUSTÍVEL LTDA',
            'LIDERCOM COMERCIO COMBUSTIVEIS RIBEIRÃO PRETO LTDA',
            'LION COMERCIO DE COMBUSTIVEIS LTDA',
            'LION COMERCIO DE COMBUSTÍVEIS LTDA',
            'LION E LION COMERCIO DE COMBUSTIVEIS LTDA',
            'LJ  COM DE COMBUSTIVEIS LTDA',
            'LLA IMIGRANTES COMERCIO VAREJISTA DE COMB',
            'LORENPOSTO COM DE COMB E SERV LTDA',
            'LUMAR COMBUSTIVEL PIQUEERI LTDA',
            'LUMMAR COMBUSTIVEIS JANDIRA LTDA',
            'LUMMAR COMBUSTIVEIS TAMBORE LTDA',
            'LUMMAR COMBUSTÍVEIS PIQUERI LTDA.',
            'LUSAN COM DE COMBUSTIVEL LTDA',
            'M MAROUBO COMERCIO DE COMBUSTIVEIS EIRELLI',
            'MAGALHAES E MAGALHAES COM COMB LUB LTDA',
            'MAGALHÃES & MAGALHÃES COM. COMBUSTÍVEIS',
            'MAGALHÃES E MAGALHÃES COMERCIO DE COMBUSTÍVEL LTDA',
            'MALAGUETA COM. VAREJISTA DE COMBUSTIVEIS LTDA',
            'MALAGUETA COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'MALAGUETA COMERCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'MALHADAL COMERCIO VAREJISTA DE COMBUSTIVEIS',
            'MAPHIGUTTI COMERCIO DE COMBUSTIVEL E DERIVADOS LTDA',
            'MARE ALTA - COMERCIO DE COMBUSTIVEIS LTDA',
            'MARE ALTA - COMERCIO DE COMBUSTIVEIS LTDA.',
            'MARE ALTA - COMERCIO DE COMBUSTÍVEIS LTDA',
            'MARE ALTA - COMERCIO DE COMBUSTÍVEL LTDA',
            'MARE ALTA COMERCIO DE COMBUSTIVEIS EIRELI',
            'MARE ALTA COMERCIO DE COMBUSTÍVEIS EIRELI',
            'MARQUES & LEÃO COM. DE COMBUST LTDA',
            'MARTINEZ & MOLINA COMERCIO DE COMBUSTIVEIS LTDA',
            'MARTINEZ E MARTINEZ COM. DE COMBUSTIVEIS LTDA.',
            'MARZOLA & FELTRIN COMERCIO DE COMBUSTÍVEL LTDA',
            'MB COMERCIO DE COMBUSTIVEIS LTDA',
            'MB COMERCIO DE COMBUSTIVEL LTDA',
            'MEDITERRANEO DO SUL COMERCIO DE COMBUSTIVEIS LTDA',
            'MILENA ELOISA POSTO DE COMBUSTIVEIS EI',
            'MILENA ELOISA VILLAVERDE POSTO DE COMBUSTIVEIS EI',
            'MILENA ELOISA VILLAVERDE POSTO DE COMBUSTÍVEIS -EI',
            'MIRAGE COMERCIO DE COMBUSTIVEIS LTDA',
            'MM COMERCIO DE COMBUSTÍVEL EIRELI',
            'MODELINHO COMERCIO DE COMBUSTIVEIS E LUBRIFICANTE',
            'MODELINHO COMERCIO DE COMBUSTIVEIS E LUBRIFICANTES',
            'MONACO BONFIM COMERCIO DE COMBUSTIVEIS LTDA',
            'MONACO COMERCIO DE COMBUSTIVEIS LTDA',
            'MONTE SANTO COMBUSTIVEIS LTDA.',
            'MONTE SANTO COMBUSTÍVEIS LTDA',
            'MONTEMOS DIST. DE COMBUSTIVEIS',
            'MR COMERCIO DE COMBUSTIVEIS EIRELI',
            'MT BATATAIS COMERCIO DE COMBUSTÍVEL LTDA',
            'NACOES COMERCIO DE COMBUSTIVEIS',
            'NAMO COMERCIO DE COMBUSTIVEIS LTDA',
            'NARDI-COM. COMBUST. DERIV. PETROLEO LTDA',
            'NAÇÕES COMERCIO DE COMBUSTIVEIS LTDA',
            'NAÇÕES COMERCIO DE COMBUSTIVEIS LTDA ME',
            'NB COM. DE COMBUSTIVEIS LTDA',
            'NOROESTE POSTO DE COMBUSTIVEIS DE VALPARAÍSO',
            'NOVA CAMPINAS COMERCIO DE COMBUSTIVEIS LTDA',
            'O BEM AMADO POSTO DE SERVIÇOS AUTOMOTIVOS E COMERCIO DE COMBU',
            'OCEANO BLUE COM. DE COMBUSTIVEIS LTDA',
            'OCEANO BLUE COM. DE COMBUSTIVEIS LTDA.',
            'OCEANO BLUE COMERCIO DE COMBUSTÍVEIS LTDA.',
            'ODALETE NATALINA MARTINS PIVA COMB.ME',
            'ODALETE NATALINA MARTINS PIVA COMBUSTÍVEIS ME',
            'OK CENTRO DE ABAST DE COMBUST LTDA',
            'OLIVEIRA E CLAUDINO COMBUSTIVEIS LTDA',
            'ORALDO JOSE TRAZZI - COMBUSTIVEIS - EPP',
            'OSTI COMERCIO DE COMBUSTIVEIS',
            'P S O COMERCIO DE COMBUSTIVEIS LTDA',
            'P S O COMERCIO DE COMBUSTÍVEIS LTDA',
            'P S U COMERCIO DE COMBUSTIVEIS LTDA',
            'P S U COMERCIO DE COMBUSTÍVEIS LTDA',
            'P V BANDEIRANTES COMERCIO DE COMB.LTDA',
            'P V BANDEIRANTES COMERCIO DE COMBUSTIVEIS LTDA',
            'P. S. COM DE COMB MALBEC LTDA',
            'P. SERV AUTO. E COM. DE COMB. BORDO LTDA',
            'P. SERV. AUTO E COM. DE COMB. BORDO LTDA.',
            'P. SERV. E CON. DE COMB. TOURIGA LTDA',
            'P.V. BANDEIRANTES COM DE COMBUSTÍVEIS LTDA',
            'P.V.BANDEIRANTES COM. DE COMBUSTIVEIS LTDA',
            'P.V.BANDEIRANTES COM. DE COMBUSTÍVEIS LTDA',
            'PACKER COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'PALACIO COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'PAX COMERCIO DE COMBUSTIVEIS EIRELI',
            'PAX COMERCIO DE COMBUSTIVEIS EIRELLI',
            'PESSEGO COM E SERV DE COMB LUB LTDA',
            'PETRO BADY COM. DE COMBUSTIVEIS LTDA',
            'PETRONEWS COMERCIO DE COMBUSTIVEIS LTDA',
            'PETRONEWS COMERCIO DE COMBUSTIVEL LTDA',
            'PETROUNI COMERCIO DE COMBUSTÍVEIS LTDA',
            'PINHO COM DE COMBUSTIVEL LTDA',
            'PINHO COM. DE COMBUSTIVEIS E DERIVADOS LTDA',
            'PINHO COMERCIO DE COMBUSTÍCEIS E DERIVADOS LTDA',
            'PINHO DE COMBUSTIVEIS E DERIVADOS LTDA',
            'POLARIS COMERCIO DE COMBUSTIVEIS LTDA',
            'PORTAL DOIS COM VAR DE COMBUSTÍVEIS LTDA',
            'PORTO ALEGRE COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTAO GETULINA COMERCIO DE COMBUSTÍVEIS LTDA',
            'POSTO ASTRO COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTO BRAS COMERCIO DE COMBUSTIVEIS E LUBRIFICANTES',
            'POSTO BRAS. COMERCIO DE COMBUSTIVEIS E LUBRIFICANTES',
            'POSTO CLARET RPA COMERCIO DE COMBUSTIVEIS RIO CLARO II LTDA',
            'POSTO COMBOIO - AUTO POSTO TATUIMAR LTDA',
            'POSTO COMBUSTIVEIS VERDE BRASIL LTDA',
            'POSTO DAS FIGUEIRAS DE COMBUSTIVEIS',
            'POSTO DAS FIGUEIRAS DE COMBUSTIVEIS DE RIBEIRÃO PRETO',
            'POSTO DAS FIGUEIRAS DE COMBUSTIVEIS DE RIBEIRÃO PRETO PRETO',
            'POSTO DE COMBUSTIVEIS DIBETH',
            'POSTO DE COMBUSTIVEIS DIBETH LTDA',
            'POSTO DE COMBUSTIVEIS E DERIVADOS DE PETROLEO  A 1 S',
            'POSTO DE COMBUSTIVEIS JEMAR LTDA',
            'POSTO DE COMBUSTIVEIS VERDE BRASILLTDA',
            'POSTO DE COMBUSTIVEIS VIA MODELO LTDA',
            'POSTO DE COMBUSTIVEL ATIBAIA SP LTDA',
            'POSTO DE COMBUSTIVEL NGM EIRELI',
            'POSTO DE COMBUSTÍVEIS SERINGUEIRAS',
            'POSTO DE COMBUSTÍVEIS VERDE BRASIL',
            'POSTO DE SERV AUTOM E COM DE COMB AEROPORTO LTDA',
            'POSTO DE SERV. AUTO E COM. DE COMB. PARIS LTDA-ME',
            'POSTO DE SERV. AUTO. E COM. DE COMB. PARIS LTDA',
            'POSTO DE SERV. AUTOMOTIV. COM. COMB. MADRI. LTDA',
            'POSTO DE SERVI.AUT.E COMB. CABERNET LTDA',
            'POSTO DE SERVICOS AUTOMOTIVOS E COMERCIO DE COMBU',
            'POSTO DE SERVICOS AUTOMOTIVOS E COMERCIO DE COMBUSTIVEIS MAD',
            'POSTO DE SERVIÇOS AUTO COM DE COMB MADRI',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COM COMBU',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COMERCIO DE COMBUSTIVEIS ALI',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COMERCIO DE COMBUSTIVEIS AMARULA LTDA',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COMERCIO DE COMBUSTÍVEL',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COMÉRCIO DE COMBU',
            'POSTO DE SERVIÇOS AUTOMOTIVOS E COMÉRCIO DE COMBUSTÍVEIS PARIS - EIRELI',
            'POSTO DE SERVIÇOS GENESIS COMERCIO DE COMBUSTIVEIS GENESIS LTDA',
            'POSTO HIROCHI & JULIA KITAGAWA COMERCIO COMBUSTIVEL',
            'POSTO KADAR COMERCIO DE COMBUSTÍVEIS LTDA',
            'POSTO MANIA REDE MM COMBUSTIVEL EIRELI',
            'POSTO MARTINELLI COMÉRCIO VAREJISTA DE COMBUSTÍVEIS',
            'POSTO MATINHA COMBUSTIVEIS - MIRASSOL LTDA',
            'POSTO MATINHA COMBUSTIVEIS - MIRASSOL LTDA.',
            'POSTO MATINHA COMBUSTIVEIS MIRASSOL LTDA',
            'POSTO MATINHA COMBUSTIVEIS MITASSOL LTDA',
            'POSTO OLIVEIRA E LIMA- POSTO DE SERVIÇOS AUTOMOTIVOS E COMÉRCIO DE COMBUSTÍVEIS',
            'POSTO PACAEMBU COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTO PALESTRA COMBUSTIVEIS - RIO PRETO LTDA',
            'POSTO PALESTRA COMBUSTÍVEIS RIO PRETO LTDA',
            'POSTO POMPEIA COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTO PONTAL DISTRIBUIDORA DE COMBUSTIVEIS LTDA',
            'POSTO PONTUAL DESTRIBUIDORA DE COMBUSTIVEL EIREL',
            'POSTO PONTUAL DISTRIBUIDORA DE COMBUSTIVEIS LTDA',
            'POSTO RECANTO ITAICI COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTO SANTA FELICIA DE COMBUSTIVEIS SÃO CARLOS LT',
            'POSTO SANTO EXPEDITO 2 IRMAOS COMERCIO DE COMBUSTIVEIS LTDA.',
            'POSTO SAO FRANCISCO VALDEMARIN 5 COM. COMB. E DER.',
            'POSTO TOTAL 9 DE JULHO COMB E CONVENIENCIA LTDA',
            'POSTO TOTAL HENRIQUE LUPO COMBUSTIVEL LTDA',
            "POSTO TRUCK'S - COMERCIO E TRANSPORTE DE COMBUSTIVEIS LTDA",
            'POSTO VENI COM DE COMBUSTÍVEIS LT',
            'POSTO VITORIA - VITORIA COMERCIO DE COMBUSTÍVEIS DE VOTUPORANGA LTDA',
            'POSTO VITORIA COMERCIO DE COMBUSTIVEIS LTDA',
            'POSTO VITORIA US COM. COMB. LTDA',
            'POSTO XV 3 HIROCHI E LUCIANA KITAGAWA COMERCIO DE COMBUSTIVE',
            'POSTO ZIL - CHAVES E TANABE REVENDA DE COMBUSTÍVEL E LUBRIFICANTES',
            'PRIMEIRO DA MALUF COMERCIO VAREJISTA DE COMBUSTIV',
            'PSG  BARRETOS COMERCIO DE COMBUSTIVEIS LTDA',
            'PSJ ANHANGUERA COMERCIO DE COMBUSTIVEIS LTDA',
            'PSJ BARRETOS COMERCIO DE COMBUSTÍVEIS LTDA',
            'PSJ GUARIBA COMERCIO DE COMBUSTIVEIS LTDA',
            'PSJ SAO SIMAO COMERCIO DE COMBUSTIVEIS LTDA',
            'PSJ SIMAO COMERCIO DE COMBUSTIVEIS LTDA',
            'PSJ SÃO SIMAO COMERCIO DE COMBUSTIVEL LTDA',
            'PSJGUARIBA COMERCIO DE COMBUSTIVEIS LTDA',
            'R C P OPERADORA DE COMBUSTIVEIS LTDA',
            'R C P OPERADORA DE COMBUSTIVEL LTDA',
            'R R COMBUSTÍVEIS LTDA',
            'R S COMERCIAL DE COMBUSTIVEIS LTDA',
            'R S COMERCIAL DE COMBUSTÍVEIS LTDA',
            'R.C.P.OPERADORA DE COMBUSTIVEIS LTDA',
            'R.S. COMERCIAL DE COMBUSTIVEIS LTDA',
            'RA COMERCIO DE COMBUSTIVEIS E DERIVADOS LTDA',
            'RADIAL LESTE COMERCIO VAREJISTA DE COMBUSTIVEL',
            'RADIAL LESTE COMERCIO VAREJISTA DE COMBUSTIVEL L',
            'RANGER COMERCIO DE COMBUSTÍVEIS LTDA',
            'RDE VIP COMÃ¿RCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'REAL COMERCIO DE COMBUSTIVEIS LTDA',
            'REAL COMERCIO DE COMBUSTIVEIS LTDA EPP',
            'REDE ATIVA SBR DE COMBUSTIVEIS LTDA',
            'REDE B B PETRO COM. DE COMBUSTIVEIS LTDA',
            'REDE BRISAS CLASS COMERCIO DE COMBUSTÁVEIS LTDA',
            'REDE BRISAS PREMIUM COMERCIO DE COMBUSTÍVEL',
            'REDE DITO COM DE COMB E DERIV DE PETROLEO LTDA',
            'REDE MARTHA COM COMBUSTIVEIS LTDA',
            'REDE MARTHA COM. COMBUSTIVEIS LT',
            'REDE MARTHA COMBUSTÍVEIS',
            'REDE MARTHA COMERCIO DE COMBUSTIVEIS LTDA',
            'REDE N BRASIL DE POSTOS DE COMBUSTIVEIS LTDA',
            'REDE ROGER DE POSTOS DE COMBUSTIVEIS LTDA',
            'REDEPOSTO BROTAS COMERCIO DE COMBUSTIVEIS LTDA',
            'REDEPOSTO QUINTANA COMERCIO DE COMBUSTIVEIS LTDA',
            'RELUDIVI JK - RIO PRETO COM DE COMBUSTIVEIS LTDA',
            'RELUDIVI LOGIST. DISTR.COM. COMBUSTIVEIS LTDA',
            'RF COMERCIO DE COMBUSTIVEIS LTDA',
            'RIBEIRO & SOUZA COMERCIO DE COMBUSTIVEIS LTDA',
            'RODO100  COMÉRCIO  DE COMBUSTIVEIS LTDA',
            'RODOANEL SUL COMERCIO VAREJISTA DE COMBUSTIVEIS',
            'RODOANEL SUL COMERCIO VAREJISTA DE COMBUSTIVEIS L',
            'RODOVIA COM. COMBUSTÍVEIS E DER. LTDA.',
            'RODOVIA COMÉRCIO DE COMBUSTIVEIS E DERIVADOS LTDA',
            'RODRIGUES & CORTEZ COMERCIAL DE COMB. LTDA',
            'RODRIGUES & CORTEZ COMERCIAL DE COMB. LTDA.',
            'RONDINELLI COMERCIO DE COMBUSTIVEIS LTDA',
            'ROSA DE SARON COMERCIO DE COMBUSTIVEIS LTDA',
            'ROSSO COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'ROSSO COMERCIO VAREJISTA DE COMBUSTÍVEIS',
            'ROSSO COMÉRCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'ROZILENI E ROZINELU COMERCIO DE COMBUSTÍVEL LTDA.',
            'ROZINELI E ROZINELI COMERCIO DE COMBUSTIVEL LTDA',
            'ROZINELI E ROZINELI COMERCIO DE COMBUSTÍVEL LTDA',
            'S A CAVALHEIRO COMBUSTIVEIS EIRELI',
            'SANGIARDI COMERCI VAREJISTA DE COMBUSTIVEIS LTD',
            'SANGIRARDI COMERCIO DE COMBUSTIVEIS LTDA',
            'SANGIRARDI COMERCIO VAREJISTA DE COMBUSTIVEIS LTD',
            'SANGIRARDI COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'SANTA BARBARA COMERCIO VAREJISTA DE COMBUSTIVEIS LTD',
            'SANTOS E ABRAO COM DE COMBUSTIVEIS LTDA',
            'SANTOS E QUEIROZ COMERCIO DE COMBUSTIVEIS LTDA',
            'SANTOS PETROL COMERCIO DE COMBUSTIVEL LTDA',
            'SAO PAULO DOIS COM VAREJO DE COMBUSTIVEIS LTDA',
            'SAO PAULO DOIS COMERIO VAREJISTA DE COMBUSTIVEIS',
            'SHEKYNAH COMERCIO DE COMBUSTIVEIS LTDA',
            'SM COMBUSTIVEIS GUARARAPES LTDA',
            'SOARES E SOARES COMERCIO DE COMBUSTIVEIS LTDA',
            'SOCIEDADE RD DE COMBUSTIVEIS E DERIVADOS LTDA',
            'SOMAR COM DE COMBUST E LUBRIFICANTES LTDA',
            'SUPER POSTO REDE ATIVA COM. COMB. LTDA.',
            'SUPORTE COM COMBUSTIVEIS E SERVICOS LTDA',
            'SUPORTE COM COMBUSTIVEIS E SERVIÇOS LTDA',
            'SUPORTE COMERCIO DE COMBUSTIVEIS E SERVIÇOS LTDA',
            'T M COM DE COMBUSTIVEIS LT',
            'TASCA MARTE COMERCIO DE COMBUSTIVEIS LTDA',
            'TASCA MARTE COMERCIO DE COMBUSTIVEL LTDA',
            'TIETE COMERCIO DE COMBUSTIVEIS E DERIVADOS LTDA',
            'TITA DA IMIGRANTES COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA M',
            'TORQUE COM VAREJISTA COMBUSTÍVEIS',
            'TORTORELLO COM. DE COMBUSTIVEIS LTDA',
            'TREVO DO PESSEGO COM SERV COMB LUB LT',
            'TREVO DO PESSEGO COMERCIO SERV COMB LTDA',
            'TRUCAO CENTRO DE ABASTECIMENTO DE COMBUSTIVEIS',
            'TRUCAO CENTRO DE ABASTECIMENTO DE COMBUSTIVEIS LTDA',
            'TRUCAO CENTRO DE ABASTECIMENTO DE COMBUSTÍVEIS LTDA',
            'TUIM COMERCIO DE COMBUSTIVEIS',
            'TUTOIA COM VERJ DE COMBUSTIVEIS LTDA',
            'TUTOIA COM. VAREJISTA DE COMBUSTIVEIS LTDA',
            'TUTOIA COM. VAREJISTA DE COMBUSTÍVEIS',
            'TUTOIA COMERCIO VAREJ. DE COMBUSTIVEIS LTDA',
            'TUTOIA COMERCIO VAREJ. DE COMBUSTIVEL',
            'TUTOIA COMERCIO VAREJISTA DE COMBUSTIVEIS',
            'TUTOIA COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'TUTOIA COMERCIO VAREJISTA DE COMBUSTIVES LTDA.',
            'TUTOIA COMERCIO VAREJISTA DE COMBUSTÍVEIS LTDA',
            'URUPES COM DE COMBUSTIVEIS E TRANSP LTDA',
            'URUPES COM. DE COMBUSTIVEIS E TRASPORT. LTDA.',
            'VALDEMARIN COM DE COMB E DER DE PETROLEO LTDA',
            'VALDEMARIN4 COM. DE COMBUSTIVEIS E DERIVADOS',
            'VALTER GAVASSA COMBUSTIVEIS LTDA',
            'VANESSA NEZZO VOLPATTI COMBUSTIVEL EIRELI',
            'VENI COM DE COMBUSTIVEIS LT',
            'VENI COM DE COMBUSTÍVEIS LTDA',
            'VENI COM. DE COMBUSTIVEIS LT',
            'VIA INDEPENDENCIA COMERCIO VAREJISTA DE COMBUSTIVEL',
            'VIA INDEPENDÊNCIA COM VAREJISTA DE COMBUSTIVEL',
            'VILELA COMERCIO DE COMBUSTÍVEIS LTDA',
            'VILLAGE COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'VITORIA COMERCIO DE COMBUSTIVEIS LTDA',
            'VITRINE COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'W R D POSTO DE COMBUSTIVEIS LTDA',
            'WERSAN COMERCIO DE COMBUSTIVEIS LTDA',
            'WINNER COMERCIO VAREJISTA DE COMBUSTIVEIS LTDA',
            'WRD POSTO DE COMBUSTIVEIS LTDA',
            'XINGU COMERCIO DE COMBUSTIVEIS LTDA',
            'XINGU COMERCIO DE COMBUSTIVEL LTDA',
            'YOSHIOKA COMERCIO DE COMBUSTIVIS LTDA',
            'ZAMBIASI COMERCIO DE COMBUSTIVEIS LTDA',
            'ZAMPOLI E ZANPOLI COM. DE COMBUSTUVEIS LTDA.',
            'ZANELLA E MATUELLA COMERCIO DE COMBUSTIVEIS LTDA'], 
            'combustivel', inplace=True)

In [93]:
ftcombustivel = df.fornecedor == 'combustivel'
dtfcombustivel = df.loc[ftcombustivel]
dtfcombustivel

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
22,2013,300098,2,20.00,1.462152e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
24,2013,300098,2,722.00,6.116149e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
26,2013,300098,2,30.00,6.116149e+13,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,combustivel
58,2013,300098,3,1282.26,6.116149e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
93,2013,300098,4,1364.28,6.116149e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
...,...,...,...,...,...,...,...
118353,2022,300655,8,44.00,1.052947e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
118356,2022,300655,8,137.13,1.291150e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
118358,2022,300655,8,183.34,3.519484e+13,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel
118384,2022,300655,9,100.00,4.750757e+12,A - COMBUSTÍVEIS E LUBRIFICANTES,combustivel


## Criando DFs Específicos

In [94]:
#criando filtros
ftfornecedor = df.fornecedor == 'hospedagem'
ftano = df.ano == 2018
ftmes = df.mes == 8

In [95]:
# criando um novo df apenas com informações filtradas
dfhospago2018 = df.loc[ftfornecedor & ftmes & ftano]

In [96]:
dfhospago2018

,ano,matricula,mes,valor,cnpj,tipo,fornecedor
4332,2018,300188,8,336.0,5.327487e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
4335,2018,300188,8,138.0,1.101320e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
28190,2018,300440,8,264.0,3.471411e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
38112,2018,300470,8,80.0,1.232690e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
55079,2018,300519,8,295.9,5.973784e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
59108,2018,300520,8,336.0,6.263718e+12,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
59127,2018,300520,8,129.0,5.768702e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
59132,2018,300520,8,445.5,9.632945e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
66030,2018,300534,8,14.0,1.486899e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem
66031,2018,300534,8,716.0,1.554179e+13,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",hospedagem


In [97]:
# descobrir valor gasto com hospedagem em agosto de 2018
sum(dfhospago2018.valor)

7227.2

## Visualização Geral

In [61]:
sorted(pd.unique(df['fornecedor']))

['1 OFICIAL DE REGISTRO DE IMOVEIS DA COMARCA DE ARARAQUARA',
 '1 OFICIAL DE REGISTRO DE IMOVEIS DE SAO PAULO',
 '1 TABELIAO DE NOTAS DA CAPITAL',
 '1 TABELIONATO DE NOTAS',
 '1 TABELIONATO DE NOTAS E ANEXOS',
 '1010 ARTESANATO EM PIZZAS LTDA',
 '1088 COND.IBIRAPUERA CENTRAL PARK',
 '11º TABELIÃO DE NOTAS',
 '123 MILHAS',
 '123 VIAGENS E TURISMO',
 '123 VIAGENS E TURISMO LTDA',
 '13 A INFORMATICA E MATERIAL DE ESCRITÓRIO LTDA.',
 '14º TABELIONATO DE NOTAS',
 '1857 COMERCEIO E EXPORTAÇÃO DE CAFÉ LTDA',
 '1857 COMERCIO E EXPORTAÇAO DE CAFE',
 '19 CARTORIO DE REGISTRO CIVIL',
 '1º OFICIAL DE REGISTRO DE IMÓVEIS DE MOGI DAS CRUZES-SP',
 '1º TABELIAO DE NOTAS E DE PROTESTOS - BEL. DENILSON DANIEL',
 '1º TABELIAO DE NOTAS E PROTESTO DE SUZANO',
 '2 M MARKETING E PLANEJAMENTO LTDA ME',
 '2 OFICIAL DE REGISTRO DE IMOVEIS, TITULOS E DOCUMENTOS E CIVIL DE PESSOA JURIDICA DE GUARULHOS',
 '2 TABELIAO DE NOTAS DA COMARCA DE OSASCO SP',
 '2 TABELIAO DE NOTAS SUZANO',
 '2 TABELIÃO DE NOTAS E DE PROTE